## Documents Extraction and Processing

In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Lets first obtain the file that maps the tickers with the CIKs

In [2]:
import requests
import json
from pathlib import Path
import os

# --- Configuration (from previous step) ---
HEADERS = {
    "User-Agent": "EdgarTutorial/1.0 (YourName your.email@domain.com)" 
}
TICKER_CIK_URL = "https://www.sec.gov/files/company_tickers.json"
OUTPUT_FILE = Path("sec_data/company_tickers.json")

# Ensure directory exists
OUTPUT_FILE.parent.mkdir(parents=True, exist_ok=True)
# ----------------------------------------

# 1. Download the JSON data
print("Downloading CIK-Ticker map...")
response = requests.get(TICKER_CIK_URL, headers=HEADERS, timeout=15)
response.raise_for_status()
raw_data = response.json() # Load into Python dictionary

# 2. Open the file and use json.dump() with indent=4
print(f"Saving JSON in readable format to {OUTPUT_FILE.absolute()}...")

# Use 'w' mode to write the file
with open(OUTPUT_FILE, 'w') as f:
    # Key Fix: The 'indent=4' parameter tells the JSON module to format the output 
    # with 4 spaces for each level of nesting, adding line breaks automatically.
    json.dump(raw_data, f, indent=4) 

print("✅ JSON saved successfully with proper line breaks and indentation.")

# --- Optional: Print a Snippet to Console (Also Pretty-Printed) ---
# If you want to print to the console instead of a file, use json.dumps()
print("\n--- Console Snippet (Pretty-Printed) ---")
# Print the first 3 key-value pairs from the dictionary
keys = list(raw_data.keys())
snippet = {k: raw_data[k] for k in keys[:3]}

# Use json.dumps() with indent=2 to format the string output
pretty_string = json.dumps(snippet, indent=2)
print(pretty_string)

Saving JSON in readable format to /home/alvar/CascadeProjects/windsurf-project/RAG/notebooks/sec_data/company_tickers.json...
✅ JSON saved successfully with proper line breaks and indentation.

--- Console Snippet (Pretty-Printed) ---
{
  "0": {
    "cik_str": 1045810,
    "ticker": "NVDA",
    "title": "NVIDIA CORP"
  },
  "1": {
    "cik_str": 1652044,
    "ticker": "GOOGL",
    "title": "Alphabet Inc."
  },
  "2": {
    "cik_str": 320193,
    "ticker": "AAPL",
    "title": "Apple Inc."
  }
}


### Vanguard Index Funds

In [10]:
import pandas as pd
from io import StringIO
from edgar import Company, set_identity
import sys
from pathlib import Path

from src.simple_rag.models.fund import FilingMetadata
from src.simple_rag.extraction.parser import BlackRockFiling


set_identity("luis.alvarez.conde@alumnos.upm.es")

ticker = "VOO"
fund = Company(ticker)
all_filings = fund.get_filings(form="N-CSR")


if all_filings:
    
    latest_date_str = max(f.report_date for f in all_filings)
    
    target_year = latest_date_str[:4]
    
    # 3. Filter: Keep ALL filings where the report_date starts with that year
    # This captures the March, June, and December reports for that fiscal year
    latest_filings = [
        f for f in all_filings 
        if f.report_date and f.report_date.startswith(target_year)
    ]
    print("Found filings: ", len(latest_filings), "for year: ", target_year)

funds_total = []
performance_funds = []
df_performance = []
for filing in latest_filings:

    print("Processing filing: ", filing.report_date)
    html_content = filing.html()
    filing_metadata = FilingMetadata(
        accession_number=filing.accession_number,
        reporting_date=filing.report_date,
        filing_date=filing.filing_date,
        form=filing.form,
        url=filing.url

    )
    parser = BlackRockFiling(html_content)
    funds = parser.get_funds()
    count = 0
    for fund in funds:
        fund.ncsr_metadata = filing_metadata
        if fund.performance_table is not None:
            performance_funds.append(fund.ticker)
            count += 1

    df_performance.append(parser.get_financial_highlights())

    print(count)
    print("Adding funds: ", len(funds))
    
    funds_total.extend(funds)

print(len(performance_funds))
print(performance_funds)
print(len(df_performance))


Found filings:  2 for year:  2024
Processing filing:  2024-12-31
Processing: Vanguard Extended Market Index Fund
Extracting context:  FY2024_C000007779Member
Tag not found:  dei:SecurityExchangeName FY2024_C000007779Member
Failed to extract tables from block:  oef:LineGraphTableTextBlock
Tables not found.
Processing: Vanguard Extended Market Index Fund
Extracting context:  FY2024_C000007782Member
Tag not found:  dei:SecurityExchangeName FY2024_C000007782Member
Failed to extract tables from block:  oef:LineGraphTableTextBlock
Tables not found.
Processing: Vanguard Extended Market Index Fund
Extracting context:  FY2024_C000007780Member
Tag not found:  dei:SecurityExchangeName FY2024_C000007780Member
Failed to extract tables from block:  oef:LineGraphTableTextBlock
Tables not found.
Processing: Vanguard Extended Market Index Fund
Extracting context:  FY2024_C000007781Member
Tag not found:  dei:SecurityExchangeName FY2024_C000007781Member
Failed to extract tables from block:  oef:LineGraph

accession_number='0001104659-25-020311' filing_date=datetime.date(2025, 3, 4) reporting_date=datetime.date(2024, 12, 31) url='https://www.sec.gov/Archives/edgar/data/36405/0001104659-25-020311-index.html' form='N-CSR'


In [ ]:
parser.print_fund_info(funds_total)\
    

Showing information of 52 funds


### 🏦 Vanguard Extended Market Index Fund

🆔 Context ID:      FY2024_C000007779Member
🎫 Ticker:          VEXMX
🏷️ Share Class:     Investor Shares
📅 Report Date:     December 31, 2024
🏛️ Sec Exchange:    N/A

--- 💰 Costs & Financials ---
Net Assets          : 111,156
Expense Ratio       : 0.19
Turnover Rate       : 11
Costs per $10k      : 21
Advisory Fees       : 1,799
Number of Holdings  : 3,485

📝 Commentary: "How did the Fund perform during the reporting period?   For the 12 months ended December 31, 2024, the Fund performed roughly in line with its benchmark index.   U.S. economic growth hovered around 3% on a year-over-year basis for much of the period,..."


**📊 Average Annual Returns**

,0,1,2,3
0,Average Annual Total Returns,NaN,NaN,NaN
1,NaN,1 Year,5 Years,10 Years
2,Investor Shares,16.76%,9.75%,9.31%
3,S&P Completion Index,16.88%,9.77%,9.33%
4,Dow Jones U.S. Total Stock Market Float Adjust...,23.88%,13.78%,12.48%


...

,Unnamed: 0,Institutional Shares,CRSP US Large Cap Index,Dow Jones U.S. Total Stock Market Float Adjusted Index
0,2014,"$5,000,000","$5,000,000","$5,000,000"
1,2015,"$5,064,717","$5,065,516","$5,090,054"
2,2015,"$5,072,527","$5,073,505","$5,096,078"
3,2015,"$4,733,990","$4,735,147","$4,725,720"
4,2015,"$5,053,400","$5,055,301","$5,022,045"
5,2016,"$5,103,277","$5,108,683","$5,067,957"
6,2016,"$5,229,161","$5,235,336","$5,200,452"
7,2016,"$5,440,308","$5,446,752","$5,431,090"
8,2016,"$5,642,490","$5,649,147","$5,655,796"
9,2017,"$5,991,191","$5,998,908","$5,983,419"


**📊 Average Annual Returns**

,Unnamed: 0,1 Year,5 Years,10 Years
0,Institutional Shares,25.12%,14.49%,13.02%
1,CRSP US Large Cap Index,25.15%,14.51%,13.05%
2,Dow Jones U.S. Total Stock Market Float Adjust...,23.88%,13.78%,12.48%


**🏗️ Sector Allocation**

,0,1,2
0,Basic Materials,1.2%,NaN
1,Consumer Discretionary,15.1%,NaN
2,Consumer Staples,4.0%,NaN
3,Energy,3.2%,NaN
4,Financials,10.8%,NaN
5,Health Care,9.8%,NaN
6,Industrials,11.3%,NaN
7,Real Estate,2.0%,NaN
8,Technology,38.0%,NaN
9,Telecommunications,2.0%,NaN


In [11]:
from src.simple_rag.models.fund import FinancialHighlights
import pandas as pd
from IPython.display import display

print(len(df_performance))

total_df = pd.concat([df_performance[0], df_performance[1]], ignore_index=True)

returns_lookup = total_df.copy()

display(returns_lookup['fund_name'].unique())

numeric_columns = ['portfolio_turnover', 'expense_ratio', 'net_assets', 
                   'nav_beginning', 'nav_end', 'net_income_ratio', 'distribution_shares']
for col in numeric_columns:
    if col in returns_lookup.columns:
        if returns_lookup[col] is not None:        
            returns_lookup[f'{col}_clean'] = (
                returns_lookup[col]
                .astype(str)
                .str.replace('%', '')
                .str.replace('$', '')
                .str.replace(',', '')
                .replace('N/A', '0')
                .replace('', '0')
                .replace('None', '0')
                .astype(float)
            )
count = 0
# Now you can efficiently match and update your funds
for fund_obj in funds_total:
    print(f"\nProcessing fund object: {fund_obj.name} - {fund_obj.share_class}")
    
    # Initialize annual returns
    if not hasattr(fund_obj, 'annual_returns') or fund_obj.annual_returns is None:
        fund_obj.annual_returns = {}
    
    # Clean the name: remove "Vanguard" and strip whitespace
    name = fund_obj.name.replace("Vanguard", "").strip()
    print(f"Cleaned name: '{name}'")
    
    # Find matching rows based on fund name
    name_matches = returns_lookup[returns_lookup['fund_name'].str.strip().str.lower() == name.lower()]
    if len(name_matches) == 0:
        print("  No name matches found for ticker: ", fund_obj.ticker)

        continue
    
    print(f"  Found {len(name_matches)} name matches")
    
    # Clean share class (remove trademark symbol)
    share_class = fund_obj.share_class
    if "™" in share_class:
        share_class = share_class.replace("™", "")
    
    # Now match share class
    share_class_matches = name_matches[
        name_matches['share_class'].str.contains(share_class, case=False, na=False, regex=False)]
    
    if len(share_class_matches) == 0:
        print(f"  No share class matches found for '{share_class}' ticker: ", fund_obj.ticker)
        print(f"  Available share classes: {name_matches['share_class'].unique()}")
        continue
    
    
    print(f"  Found {len(share_class_matches)} matching records")
    count += 1
    # Add all matching returns
    for _, row in share_class_matches.iterrows():
        year = str(row['year'])
        
       
        highlights = FinancialHighlights(
            turnover=row.get('portfolio_turnover_clean', 0),
            expense_ratio=row.get('expense_ratio_clean', 0),
            total_return=row['total_return'],
            net_assets=row.get('net_assets_clean', 0),
            net_assets_value_begining=row.get('nav_beginning_clean', 0),
            net_assets_value_end=row.get('nav_end_clean', 0),
            net_income_ratio=row.get('net_income_ratio_clean', 0.0)
        )
        
        fund_obj.financial_highlights[year] = highlights
        print(f"  {year}: Total Return = {highlights.total_return}%, Expense Ratio = {highlights.expense_ratio}%, Net Assets = {highlights.net_assets}, Net Income Ratio = {highlights.net_income_ratio}, Turnover = {highlights.turnover}, Net Assets Value Begining = {highlights.net_assets_value_begining}, Net Assets Value End = {highlights.net_assets_value_end}")
print("count: ",count)
print("Total funds: ",len(funds_total))
    

2


array(['Small-Cap Index Fund', 'Small-Cap Growth Index Fund',
       'Small-Cap Value Index Fund', 'Extended Market Index Fund',
       'Mid-Cap Index Fund', 'Mid-Cap Growth Index Fund',
       'Mid-Cap Value Index Fund', 'Total Stock Market Index Fund',
       '500 Index Fund', 'Growth Index Fund', 'Value Index Fund',
       'Large-Cap Index Fund'], dtype=object)


Processing fund object: Vanguard Extended Market Index Fund - ShareClassType.INVESTOR
Cleaned name: 'Extended Market Index Fund'
  Found 30 name matches
  Found 5 matching records
  2024: Total Return = 16.76%, Expense Ratio = 0.19%, Net Assets = 195.0, Net Income Ratio = 1.09, Turnover = 11.0, Net Assets Value Begining = 124.78, Net Assets Value End = 144.2
  2023: Total Return = 25.22%, Expense Ratio = 0.19%, Net Assets = 232.0, Net Income Ratio = 1.28, Turnover = 11.0, Net Assets Value Begining = 100.93, Net Assets Value End = 124.78
  2022: Total Return = -26.56%, Expense Ratio = 0.19%, Net Assets = 229.0, Net Income Ratio = 1.14, Turnover = 11.0, Net Assets Value Begining = 138.8, Net Assets Value End = 100.93
  2021: Total Return = 12.31%, Expense Ratio = 0.19%, Net Assets = 399.0, Net Income Ratio = 0.87, Turnover = 19.0, Net Assets Value Begining = 124.83, Net Assets Value End = 138.8
  2020: Total Return = 32.04%, Expense Ratio = 0.19%, Net Assets = 454.0, Net Income Ratio = 

In [12]:
import sys
%reload_ext autoreload
from src.simple_rag.extraction.parser import compute_annual_returns

for fund in funds_total:
    if fund.ticker in performance_funds:
        returns = compute_annual_returns(fund.performance_table)
        print("\nFinal Annual Returns:")
        fund.annual_returns = returns
        print(f"  {fund.ticker}: {returns}")
        for year, return_ in returns.items():
            print(fund.financial_highlights.keys())
            if year not in fund.financial_highlights.keys():
                new_highlight = FinancialHighlights(
                year=int(year),
                total_return=return_,
                turnover=0.0,
                expense_ratio=0.0,
                net_assets=0.0,
                net_assets_value_begining=0.0,
                net_assets_value_end=0.0,
                net_income_ratio=0.0
                )
                fund.financial_highlights[year] = new_highlight
                print(f"    {year}: {new_highlight}")

Detected format: Year (YYYY)
Found years: [np.int64(2014), np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024)]
  2015 Return: $10,000.00 -> $10,125.00 = 1.25%
  2016 Return: $10,125.00 -> $11,321.00 = 11.81%
  2017 Return: $11,321.00 -> $13,774.00 = 21.67%
  2018 Return: $13,774.00 -> $13,151.00 = -4.52%
  2019 Return: $13,151.00 -> $17,271.00 = 31.33%
  2020 Return: $17,271.00 -> $20,423.00 = 18.25%
  2021 Return: $20,423.00 -> $26,250.00 = 28.53%
  2022 Return: $26,250.00 -> $21,465.00 = -18.23%
  2023 Return: $21,465.00 -> $27,069.00 = 26.11%
  2024 Return: $27,069.00 -> $33,794.00 = 24.84%

Final Annual Returns:
  VFINX: {'2015': 1.25, '2016': 11.81, '2017': 21.67, '2018': -4.52, '2019': 31.33, '2020': 18.25, '2021': 28.53, '2022': -18.23, '2023': 26.11, '2024': 24.84}
dict_keys(['2024', '2023', '2022', '2021', '2020'])
    2015: turnover=0.0 expense_ratio=0.0 total_return=1.

/home/luis/Desktop/code/RAG/src/simple_rag/extraction/parser.py:69: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['parsed_date'] = pd.to_datetime(df[date_col], errors='coerce')


## Vanguard World Fund

In [13]:
import pandas as pd
from io import StringIO
import sys
from pathlib import Path
RAG_DIR = Path("/home/alvar/CascadeProjects/windsurf-project/RAG")
if str(RAG_DIR) not in sys.path:
    sys.path.insert(0, str(RAG_DIR))
%reload_ext autoreload
from src.simple_rag.extraction.parser import BlackRockFiling
from edgar import set_identity, Company

from src.simple_rag.models.fund import FilingMetadata

set_identity("luis.alvarez.conde@alumnos.upm.es")

ticker = "MGK"
fund = Company(ticker)
all_filings = fund.get_filings(form="N-CSR")


if all_filings:
    # 1. Find the most recent date in the entire history (e.g., "2024-12-31")
    latest_date_str = max(f.report_date for f in all_filings)
    
    # 2. Extract just the YEAR (e.g., "2024")
    target_year = latest_date_str[:4]
    
    # 3. Filter: Keep ALL filings where the report_date starts with that year
    # This captures the March, June, and December reports for that fiscal year
    latest_filings = [
        f for f in all_filings 
        if f.report_date and f.report_date.startswith(target_year)
    ]
    target_year = "2024"
    filings2 = sorted(
        [f for f in all_filings if f.report_date and f.report_date.startswith(target_year)],
        key=lambda f: f.report_date,
        reverse=True
    )

    latest_filings.append(filings2[0])
    print("Found filings: ", len(latest_filings), "for year: ", target_year)



performance_funds = []
df_performance = []
world_funds = set()

abort = False
for filing in latest_filings:

    html_content = filing.html()
    
    parser = BlackRockFiling(html_content)
    funds = parser.get_funds()
    count = 0
    filing_metadata = FilingMetadata(
        accession_number=filing.accession_number,
        reporting_date=filing.report_date,
        filing_date=filing.filing_date,
        form=filing.form,
        url=filing.url

    )
    for fund in funds:
        if fund.performance_table is not None:
            if fund.ticker not in performance_funds:
                performance_funds.append(fund.ticker)
                count += 1
        if fund.ticker not in world_funds:
            world_funds.add(fund.ticker)
        else:
            print("Exiting filing, repeated ticker found: ", fund.ticker)
            abort = True
            break
        fund.ncsr_metadata = filing_metadata 
    if not abort:

        df_performance.append(parser.get_financial_highlights())
        print(count)
        print("Adding funds: ", len(funds))
        funds_total.extend(funds)

print("Total world funds added: ", len(world_funds))
print(len(performance_funds))
print(performance_funds)

print(len(df_performance))


Found filings:  4 for year:  2024
Processing: Mega Cap Growth Index Fund
Extracting context:  From2024-10-01to2025-09-30_C000055216Member
Processing: Mega Cap Growth Index Fund
Extracting context:  From2024-10-01to2025-09-30_C000055215Member
Tag not found:  dei:SecurityExchangeName From2024-10-01to2025-09-30_C000055215Member
2
Adding funds:  2
Processing: Vanguard Extended Duration Treasury Index Fund
Extracting context:  FY2025_C000051981Member
Tag not found:  dei:SecurityExchangeName FY2025_C000051981Member
Failed to extract tables from block:  oef:LineGraphTableTextBlock
Tables not found.
Processing: Vanguard Extended Duration Treasury Index Fund
Extracting context:  FY2025_C000051979Member
Tag not found:  dei:SecurityExchangeName FY2025_C000051979Member
Failed to extract tables from block:  oef:LineGraphTableTextBlock
Tables not found.
Processing: Vanguard ESG U.S. Stock ETF
Extracting context:  FY2025_C000204567Member
Tag not found:  dei:SecurityExchangeName FY2025_C000204567Membe

In [14]:
from src.simple_rag.models.fund import FinancialHighlights
import pandas as pd
from IPython.display import display

print(len(df_performance))

total_df = pd.concat([df_performance[0], df_performance[1], df_performance[2]], ignore_index=True)

returns_lookup = total_df.copy()

# Optional: Clean the total_return column (remove % sign if needed)
print(returns_lookup.head())
display(returns_lookup['fund_name'].unique())

numeric_columns = ['portfolio_turnover', 'expense_ratio', 'net_assets', 
                   'nav_beginning', 'nav_end', 'net_income_ratio', 'distribution_shares']


for col in numeric_columns:
    if col in returns_lookup.columns:
        if returns_lookup[col] is not None:  
            try:      
                returns_lookup[f'{col}_clean'] = (
                    returns_lookup[col]
                    .astype(str)
                    .str.replace('%', '')
                    .str.replace('$', '')
                    .str.replace(',', '')
                    .replace('N/A', '0')
                    .replace('', '0')
                    .replace('None', '0')
                    .astype(float)
                )
            except Exception as e:
                print(f"Error cleaning column '{col}': {str(e)}")
                print(returns_lookup[col].to_string())
count = 0
# Now you can efficiently match and update your funds
for fund_obj in funds_total:
    print(f"\nProcessing fund object: {fund_obj.name} - {fund_obj.share_class}")
    
    # Initialize annual returns
    if not hasattr(fund_obj, 'annual_returns') or fund_obj.annual_returns is None:
        fund_obj.annual_returns = {}
    
    # Clean the name: remove "Vanguard" and strip whitespace
    name = fund_obj.name.replace("Vanguard", "").strip()
    print(f"Cleaned name: '{name}'")
    
    if "™" in name:
        name = name.replace("™", "")
    elif "®" in name:
        name = name.replace("®", "")
    # Find matching rows based on fund name
    name_matches = returns_lookup[returns_lookup['fund_name'].str.strip().str.lower() == name.lower()]
    if len(name_matches) == 0:
        print("  No name matches found for ticker: ", fund_obj.ticker)
        continue
    
    print(f"  Found {len(name_matches)} name matches")
    
    # Clean share class (remove trademark symbol)
    share_class = fund_obj.share_class
    if "™" in share_class:
        share_class = share_class.replace("™", "")
    
    # Now match share class
    share_class_matches = name_matches[
        name_matches['share_class'].str.contains(share_class, case=False, na=False, regex=False)]
    
    if len(share_class_matches) == 0:
        print(f"  No share class matches found for '{share_class}' ticker: ", fund_obj.ticker)
        print(f"  Available share classes: {name_matches['share_class'].unique()}")
        continue
    elif len(share_class_matches) > 5:
        print("  More than 5 share class matches found:")
        print(share_class_matches)
    
    
    print(f"  Found {len(share_class_matches)} matching records")
    count += 1
    # Add all matching returns
    for _, row in share_class_matches.iterrows():
        year = str(row['year'])
        
       
        highlights = FinancialHighlights(
            turnover=row.get('portfolio_turnover_clean', 0),
            expense_ratio=row.get('expense_ratio_clean', 0),
            total_return=row['total_return'],
            net_assets=row.get('net_assets_clean', 0),
            net_assets_value_begining=row.get('nav_beginning_clean', 0),
            net_assets_value_end=row.get('nav_end_clean', 0),
            net_income_ratio=row.get('net_income_ratio_clean', 0.0)
        )
        
        fund_obj.financial_highlights[year] = highlights
        print(f"  {year}: Total Return = {highlights.total_return}%, Expense Ratio = {highlights.expense_ratio}%, Net Assets = {highlights.net_assets}, Net Income Ratio = {highlights.net_income_ratio}, Turnover = {highlights.turnover}, Net Assets Value Begining = {highlights.net_assets_value_begining}, Net Assets Value End = {highlights.net_assets_value_end}")
print("count: ",count)
print("Total funds: ",world_funds)
    

3
                    fund_name share_class  year  net_assets  nav_beginning  \
0  Mega Cap Growth Index Fund  ETF Shares  2025     31195.0         321.87   
1  Mega Cap Growth Index Fund  ETF Shares  2024     22954.0         314.83   
2  Mega Cap Growth Index Fund  ETF Shares  2024     21996.0         241.25   
3  Mega Cap Growth Index Fund  ETF Shares  2023     14376.0         195.20   
4  Mega Cap Growth Index Fund  ETF Shares  2022     11168.0         248.50   

   nav_end  total_return  expense_ratio  net_income_ratio  portfolio_turnover  \
0   402.45         25.58           0.07              0.42                14.0   
1   321.87          2.35           0.07              0.40                 6.0   
2   314.83         31.16           0.07              0.51                14.0   
3   241.25         24.39           0.07              0.62                 7.0   
4   195.20        -21.08           0.07              0.51                 5.0   

  distribution_shares  
0                N

array(['Mega Cap Growth Index Fund',
       'Extended Duration Treasury Index Fund', 'ESG U.S. Stock ETF',
       'ESG International Stock ETF', 'Global Wellington Fund',
       'Global Wellesley Income Fund', 'ESG U.S. Corporate Bond ETF',
       'U.S. Growth Fund', 'International Growth Fund',
       'FTSE Social Index Fund', 'Communication Services Index Fund',
       'Consumer Discretionary Index Fund', 'Consumer Staples Index Fund',
       'Energy Index Fund', 'Financials Index Fund',
       'Health Care Index Fund', 'Industrials Index Fund',
       'Information Technology Index Fund', 'Materials Index Fund',
       'Utilities Index Fund', 'Mega Cap Index Fund',
       'Mega Cap Value Index Fund'], dtype=object)


Processing fund object: Vanguard Extended Market Index Fund - ShareClassType.INVESTOR
Cleaned name: 'Extended Market Index Fund'
  No name matches found for ticker:  VEXMX

Processing fund object: Vanguard Extended Market Index Fund - ShareClassType.ETF
Cleaned name: 'Extended Market Index Fund'
  No name matches found for ticker:  VXF

Processing fund object: Vanguard Extended Market Index Fund - ShareClassType.ADMIRAL
Cleaned name: 'Extended Market Index Fund'
  No name matches found for ticker:  VEXAX

Processing fund object: Vanguard Extended Market Index Fund - ShareClassType.INSTITUTIONAL
Cleaned name: 'Extended Market Index Fund'
  No name matches found for ticker:  VIEIX

Processing fund object: Vanguard Extended Market Index Fund - ShareClassType.INSTITUTIONAL_PLUS
Cleaned name: 'Extended Market Index Fund'
  No name matches found for ticker:  VEMPX

Processing fund object: Vanguard Extended Market Index Fund - ShareClassType.INSTITUTIONAL_SELECT
Cleaned name: 'Extended Marke

In [15]:
import sys
%reload_ext autoreload
sys.path.append('../src')


from simple_rag.extraction.parser import compute_annual_returns

for fund in funds_total:
    if fund.ticker in performance_funds:
        returns = compute_annual_returns(fund.performance_table)
        print("\nFinal Annual Returns:")
        fund.annual_returns = returns
        print(f"  {fund.ticker}: {returns}")
        for year, return_ in returns.items():
            print(fund.financial_highlights.keys())
            if year not in fund.financial_highlights.keys():
                new_highlight = FinancialHighlights(
                year=int(year),
                total_return=return_,
                turnover=0.0,
                expense_ratio=0.0,
                net_assets=0.0,
                net_assets_value_begining=0.0,
                net_assets_value_end=0.0,
                net_income_ratio=0.0
                )
                fund.financial_highlights[year] = new_highlight
                print(f"    {year}: {new_highlight}")

Detected format: Year (YYYY)
Found years: [np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024), np.int64(2025)]
  2016 Return: $10,724.00 -> $11,409.00 = 6.39%
  2017 Return: $11,409.00 -> $14,772.00 = 29.48%
  2018 Return: $14,772.00 -> $14,349.00 = -2.86%
  2019 Return: $14,349.00 -> $19,736.00 = 37.54%
  2020 Return: $19,736.00 -> $27,826.00 = 40.99%
  2021 Return: $27,826.00 -> $35,753.00 = 28.49%
  2022 Return: $35,753.00 -> $23,755.00 = -33.56%
  2023 Return: $23,755.00 -> $36,004.00 = 51.56%
  2024 Return: $36,004.00 -> $47,873.00 = 32.97%
  2025 Return: $47,873.00 -> $56,289.00 = 17.58%

Final Annual Returns:
  MGK: {'2016': 6.39, '2017': 29.48, '2018': -2.86, '2019': 37.54, '2020': 40.99, '2021': 28.49, '2022': -33.56, '2023': 51.56, '2024': 32.97, '2025': 17.58}
dict_keys(['2025', '2024', '2023', '2022', '2021'])
    2016: turnover=0.0 expense_ratio=0.0 total_return=6.39

In [16]:
import pickle
from pathlib import Path
import sys

# Add RAG directory to path
RAG_DIR = Path("/home/alvar/CascadeProjects/windsurf-project/RAG")
if str(RAG_DIR) not in sys.path:
    sys.path.insert(0, str(RAG_DIR))

# Define pickle file path
PKL_PATH = Path("./funds_backup_metadata.pkl")

print("Current working directory:", Path.cwd())
print("PKL_PATH resolves to:", PKL_PATH.resolve())

# Save to pickle file
try:
    with PKL_PATH.open("wb") as f:
        pickle.dump(funds_total, f)
    
    print(f"Successfully saved {len(funds_total)} funds to pickle file")
    print(f"File size: {PKL_PATH.stat().st_size / 1024:.2f} KB")
    
except Exception as e:
    print(f"Error saving to pickle file: {e}")

Current working directory: /home/luis/Desktop/code/RAG/notebooks
PKL_PATH resolves to: /home/luis/Desktop/code/RAG/notebooks/funds_backup_metadata.pkl
Successfully saved 93 funds to pickle file
File size: 403.82 KB


## Vanguard Specialized Funds

In [17]:
import pandas as pd
from io import StringIO

import sys
from pathlib import Path
from src.simple_rag.extraction.parser import BlackRockFiling
from edgar import set_identity, Company

from src.simple_rag.models.fund import FilingMetadata

set_identity("luis.alvarez.conde@alumnos.upm.es")

ticker = "VDIGX"
fund = Company(ticker)
all_filings = fund.get_filings(form="N-CSR")


if all_filings:
    # 1. Find the most recent date in the entire history (e.g., "2024-12-31")
    latest_date_str = max(f.report_date for f in all_filings)
    
    # 2. Extract just the YEAR (e.g., "2024")
    target_year = latest_date_str[:4]
    
    # 3. Filter: Keep ALL filings where the report_date starts with that year
    # This captures the March, June, and December reports for that fiscal year
    latest_filings = [
        f for f in all_filings 
        if f.report_date and f.report_date.startswith(target_year)
    ]
    
    print("Found filings: ", len(latest_filings), "for year: ", target_year)



performance_funds = []
specialized_funds = set()
df_performance = []
abort = False
for filing in latest_filings:

    html_content = filing.html()
    
    parser = BlackRockFiling(html_content)
    funds = parser.get_funds()
    count = 0
    filing_metadata = FilingMetadata(
        accession_number=filing.accession_number,
        reporting_date=filing.report_date,
        filing_date=filing.filing_date,
        form=filing.form,
        url=filing.url

    )
    for fund in funds:
        if fund.performance_table is not None:
            if fund.ticker not in performance_funds: 
                performance_funds.append(fund.ticker)
                count += 1
        if fund.ticker not in specialized_funds:
            specialized_funds.add(fund.ticker)
        else:
            print("Exiting filing, repeated ticker found: ", fund.ticker)
            abort = True
            break
        fund.ncsr_metadata = filing_metadata
    if abort:
        break
    df_performance.append(parser.get_financial_highlights())

    print(count)
    print("Adding funds: ", len(funds))
    funds_total.extend(funds)

print("Specialized funds: ", len(specialized_funds))
print(len(performance_funds))
print(performance_funds)
print(len(df_performance))



Found filings:  2 for year:  2025
Processing: Dividend Growth Fund
Extracting context:  From2024-02-01to2025-01-31_C000008004Member
Tag not found:  dei:SecurityExchangeName From2024-02-01to2025-01-31_C000008004Member
Processing: Energy Fund
Extracting context:  From2024-02-01to2025-01-31_C000008005Member
Tag not found:  dei:SecurityExchangeName From2024-02-01to2025-01-31_C000008005Member
Processing: Energy Fund
Extracting context:  From2024-02-01to2025-01-31_C000008006Member
Tag not found:  dei:SecurityExchangeName From2024-02-01to2025-01-31_C000008006Member
Processing: Health Care Fund
Extracting context:  From2024-02-01to2025-01-31_C000008007Member
Tag not found:  dei:SecurityExchangeName From2024-02-01to2025-01-31_C000008007Member
Processing: Health Care Fund
Extracting context:  From2024-02-01to2025-01-31_C000008008Member
Tag not found:  dei:SecurityExchangeName From2024-02-01to2025-01-31_C000008008Member
Processing: Dividend Appreciation Index Fund
Extracting context:  From2024-02

In [18]:
from src.simple_rag.models.fund import FinancialHighlights
import pandas as pd
from IPython.display import display

print(len(df_performance))

total_df = pd.concat([df_performance[0], df_performance[1]], ignore_index=True)

returns_lookup = total_df.copy()

# Optional: Clean the total_return column (remove % sign if needed)
print(returns_lookup.head())
display(returns_lookup['fund_name'].unique())

numeric_columns = ['portfolio_turnover', 'expense_ratio', 'net_assets', 
                   'nav_beginning', 'nav_end', 'net_income_ratio', 'distribution_shares']


for col in numeric_columns:
    if col in returns_lookup.columns:
        if returns_lookup[col] is not None:  
            try:      
                returns_lookup[f'{col}_clean'] = (
                    returns_lookup[col]
                    .astype(str)
                    .str.replace('%', '')
                    .str.replace('$', '')
                    .str.replace(',', '')
                    .replace('N/A', '0')
                    .replace('', '0')
                    .replace('None', '0')
                    .astype(float)
                )
            except Exception as e:
                print(f"Error cleaning column '{col}': {str(e)}")
                print(returns_lookup[col].to_string())
count = 0
# Now you can efficiently match and update your funds
for fund_obj in funds_total:
    if fund_obj.ticker not in specialized_funds:
        continue
    print(f"\nProcessing fund object: {fund_obj.name} - {fund_obj.share_class}")
    
    # Initialize annual returns
    if not hasattr(fund_obj, 'annual_returns') or fund_obj.annual_returns is None:
        fund_obj.annual_returns = {}
    
    # Clean the name: remove "Vanguard" and strip whitespace
    name = fund_obj.name.replace("Vanguard", "").strip()
    print(f"Cleaned name: '{name}'")
    
    if "™" in name:
        name = name.replace("™", "")
    elif "®" in name:
        name = name.replace("®", "")
    # Find matching rows based on fund name
    name_matches = returns_lookup[returns_lookup['fund_name'].str.strip().str.lower() == name.lower()]
    if len(name_matches) == 0:
        print("  No name matches found for ticker: ", fund_obj.ticker)
        continue
    
    print(f"  Found {len(name_matches)} name matches")
    
    # Clean share class (remove trademark symbol)
    share_class = fund_obj.share_class
    if "™" in share_class:
        share_class = share_class.replace("™", "")
    
    # Now match share class
    share_class_matches = name_matches[
        name_matches['share_class'].str.contains(share_class, case=False, na=False, regex=False)]
    
    if len(share_class_matches) == 0:
        print(f"  No share class matches found for '{share_class}' ticker: ", fund_obj.ticker)
        print(f"  Available share classes: {name_matches}")
        print(f"  Found {len(share_class_matches)} matching records")
        count += 1
        # Add all matching returns
        for _, row in share_class_matches.iterrows():
            year = str(row['year'])
            highlights = FinancialHighlights(
                turnover=row.get('portfolio_turnover_clean', 0),
                expense_ratio=row.get('expense_ratio_clean', 0),
                total_return=row['total_return'],
                net_assets=row.get('net_assets_clean', 0),
                net_assets_value_begining=row.get('nav_beginning_clean', 0),
                net_assets_value_end=row.get('nav_end_clean', 0),
                net_income_ratio=row.get('net_income_ratio_clean', 0.0)
            )
            
            fund_obj.financial_highlights[year] = highlights
            print(f"  {year}: Total Return = {highlights.total_return}%, Expense Ratio = {highlights.expense_ratio}%, Net Assets = {highlights.net_assets}, Net Income Ratio = {highlights.net_income_ratio}, Turnover = {highlights.turnover}, Net Assets Value Begining = {highlights.net_assets_value_begining}, Net Assets Value End = {highlights.net_assets_value_end}")
        continue
    elif len(share_class_matches) > 5:
        print("  More than 5 share class matches found:")
        print(share_class_matches)
    
    
    print(f"  Found {len(share_class_matches)} matching records")
    count += 1
    # Add all matching returns
    for _, row in share_class_matches.iterrows():
        year = str(row['year'])
        highlights = FinancialHighlights(
            turnover=row.get('portfolio_turnover_clean', 0),
            expense_ratio=row.get('expense_ratio_clean', 0),
            total_return=row['total_return'],
            net_assets=row.get('net_assets_clean', 0),
            net_assets_value_begining=row.get('nav_beginning_clean', 0),
            net_assets_value_end=row.get('nav_end_clean', 0),
            net_income_ratio=row.get('net_income_ratio_clean', 0.0)
        )
        
        fund_obj.financial_highlights[year] = highlights
        print(f"  {year}: Total Return = {highlights.total_return}%, Expense Ratio = {highlights.expense_ratio}%, Net Assets = {highlights.net_assets}, Net Income Ratio = {highlights.net_income_ratio}, Turnover = {highlights.turnover}, Net Assets Value Begining = {highlights.net_assets_value_begining}, Net Assets Value End = {highlights.net_assets_value_end}")
print("count: ",count)
print("Total funds: ",specialized_funds)
    

2
              fund_name share_class  year  net_assets  nav_beginning  nav_end  \
0  Dividend Growth Fund        None  2025     50424.0          37.76    37.14   
1  Dividend Growth Fund        None  2024     52553.0          35.42    37.76   
2  Dividend Growth Fund        None  2023     53452.0          37.85    35.42   
3  Dividend Growth Fund        None  2022     54186.0          31.82    37.85   
4  Dividend Growth Fund        None  2021     45099.0          30.63    31.82   

   total_return  expense_ratio  net_income_ratio  portfolio_turnover  \
0         10.20           0.22              1.68                16.0   
1          9.11           0.29              1.74                 9.0   
2         -0.76           0.30              1.68                11.0   
3         25.66           0.27              1.56                15.0   
4          7.03           0.26              1.85                15.0   

  distribution_shares  
0                None  
1                None  
2     

array(['Dividend Growth Fund', 'Energy Fund', 'Health Care Fund',
       'Dividend Appreciation Index Fund', 'Real Estate Index Fund',
       'Real Estate II Index Fund', 'Global Capital Cycles Fund',
       'Global ESG Select Stock Fund'], dtype=object)


Processing fund object: Dividend Growth Fund - ShareClassType.INVESTOR
Cleaned name: 'Dividend Growth Fund'
  Found 5 name matches
  No share class matches found for 'ShareClassType.INVESTOR' ticker:  VDIGX
  Available share classes:               fund_name share_class  year  net_assets  nav_beginning  nav_end  \
0  Dividend Growth Fund        None  2025     50424.0          37.76    37.14   
1  Dividend Growth Fund        None  2024     52553.0          35.42    37.76   
2  Dividend Growth Fund        None  2023     53452.0          37.85    35.42   
3  Dividend Growth Fund        None  2022     54186.0          31.82    37.85   
4  Dividend Growth Fund        None  2021     45099.0          30.63    31.82   

   total_return  expense_ratio  net_income_ratio  portfolio_turnover  \
0         10.20           0.22              1.68                16.0   
1          9.11           0.29              1.74                 9.0   
2         -0.76           0.30              1.68              

In [19]:
import sys
%reload_ext autoreload
sys.path.append('../src')


from simple_rag.extraction.parser import compute_annual_returns

for fund in funds_total:
    if fund.ticker in performance_funds:
        returns = compute_annual_returns(fund.performance_table)
        print("\nFinal Annual Returns:")
        fund.annual_returns = returns
        print(f"  {fund.ticker}: {returns}")
        for year, return_ in returns.items():
            print(fund.financial_highlights.keys())
            if year not in fund.financial_highlights.keys():
                new_highlight = FinancialHighlights(
                year=int(year),
                total_return=return_,
                turnover=0.0,
                expense_ratio=0.0,
                net_assets=0.0,
                net_assets_value_begining=0.0,
                net_assets_value_end=0.0,
                net_income_ratio=0.0
                )
                fund.financial_highlights[year] = new_highlight
                print(f"    {year}: {new_highlight}")

Detected format: Year (YYYY)
Found years: [np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024), np.int64(2025)]
  2016 Return: $10,619.00 -> $10,874.00 = 2.40%
  2017 Return: $10,874.00 -> $12,860.00 = 18.26%
  2018 Return: $12,860.00 -> $14,143.00 = 9.98%
  2019 Return: $14,143.00 -> $16,879.00 = 19.35%
  2020 Return: $16,879.00 -> $17,545.00 = 3.95%
  2021 Return: $17,545.00 -> $23,646.00 = 34.77%
  2022 Return: $23,646.00 -> $22,953.00 = -2.93%
  2023 Return: $22,953.00 -> $23,134.00 = 0.79%
  2024 Return: $23,134.00 -> $28,121.00 = 21.56%
  2025 Return: $28,121.00 -> $28,555.00 = 1.54%

Final Annual Returns:
  VDIGX: {'2016': 2.4, '2017': 18.26, '2018': 9.98, '2019': 19.35, '2020': 3.95, '2021': 34.77, '2022': -2.93, '2023': 0.79, '2024': 21.56, '2025': 1.54}
dict_keys([])
    2016: turnover=0.0 expense_ratio=0.0 total_return=2.4 net_assets=0.0 net_assets_value_begining=0.0 ne

In [20]:
import pickle
from pathlib import Path
import sys

# Add RAG directory to path
RAG_DIR = Path("/home/alvar/CascadeProjects/windsurf-project/RAG")
if str(RAG_DIR) not in sys.path:
    sys.path.insert(0, str(RAG_DIR))

# Define pickle file path
PKL_PATH = Path("./funds_backup_metadata.pkl")

print("Current working directory:", Path.cwd())
print("PKL_PATH resolves to:", PKL_PATH.resolve())

# Save to pickle file
try:
    with PKL_PATH.open("wb") as f:
        pickle.dump(funds_total, f)
    
    print(f"Successfully saved {len(funds_total)} funds to pickle file")
    print(f"File size: {PKL_PATH.stat().st_size / 1024:.2f} KB")
    
except Exception as e:
    print(f"Error saving to pickle file: {e}")

Current working directory: /home/luis/Desktop/code/RAG/notebooks
PKL_PATH resolves to: /home/luis/Desktop/code/RAG/notebooks/funds_backup_metadata.pkl
Successfully saved 107 funds to pickle file
File size: 461.90 KB


In [ ]:
import pickle
from pathlib import Path
from dataclasses import is_dataclass, asdict
import pandas as pd
import sys
from pathlib import Path
RAG_DIR = Path("/home/alvar/CascadeProjects/windsurf-project/RAG")
if str(RAG_DIR) not in sys.path:
    sys.path.insert(0, str(RAG_DIR))


PKL_PATH = Path("./funds_backup.pkl")
print("Current working directory:", Path.cwd())
print("PKL_PATH resolves to:", PKL_PATH.resolve())
with PKL_PATH.open("rb") as f:
    funds_total = pickle.load(f)

print(f"Loaded {len(funds_total)} funds from pickle file")

Current working directory: /home/alvar/CascadeProjects/windsurf-project/RAG/notebooks
PKL_PATH resolves to: /home/alvar/CascadeProjects/windsurf-project/RAG/notebooks/funds_backup.pkl
Loaded 107 funds from pickle file


## Vanguard Whitehall Funds

In [21]:
import pandas as pd
from io import StringIO
import sys
from pathlib import Path
from src.simple_rag.extraction.parser import BlackRockFiling
from edgar import set_identity, Company

from src.simple_rag.models.fund import FundData

set_identity("luis.alvarez.conde@alumnos.upm.es")
ticker = "VMGRX"
fund = Company(ticker)
all_filings = fund.get_filings(form="N-CSR")


if all_filings:
    # 1. Find the most recent date in the entire history (e.g., "2024-12-31")
    latest_date_str = max(f.report_date for f in all_filings)
    
    # 2. Extract just the YEAR (e.g., "2024")
    target_year = latest_date_str[:4]
    
    # 3. Filter: Keep ALL filings where the report_date starts with that year
    # This captures the March, June, and December reports for that fiscal year
    latest_filings = [
        f for f in all_filings 
        if f.report_date and f.report_date.startswith(target_year)
    ]
    
    print("Found filings: ", len(latest_filings), "for year: ", target_year)



performance_funds = []
whitehall_funds = set()
df_performance = []
abort = False

for filing in latest_filings:

    html_content = filing.html()
    
    parser = BlackRockFiling(html_content)
    funds = parser.get_funds()
    count = 0
    filing_metadata = FilingMetadata(
        accession_number=filing.accession_number,
        reporting_date=filing.report_date,
        filing_date=filing.filing_date,
        form=filing.form,
        url=filing.url

    )
    for fund in funds:
        if fund.performance_table is not None:
            if fund.ticker not in performance_funds: 
                performance_funds.append(fund.ticker)
                count += 1

        if fund.ticker not in whitehall_funds:
            whitehall_funds.add(fund.ticker)
        else:
            print("Exiting filing, repeated ticker found: ", fund.ticker)
            abort = True
            break
        fund.ncsr_metadata = filing_metadata
    if abort:
        break

    df_performance.append(parser.get_financial_highlights())

    print(count)
    print("Adding funds: ", len(funds))
    funds_total.extend(funds)

print("Whitehall funds: ", len(whitehall_funds))
print(len(performance_funds))
print(whitehall_funds)
print(len(df_performance))


Found filings:  2 for year:  2025
Processing: Mid-Cap Growth Fund
Extracting context:  From2024-11-01to2025-10-31_C000012166Member
Tag not found:  dei:SecurityExchangeName From2024-11-01to2025-10-31_C000012166Member
Processing: Selected Value Fund
Extracting context:  From2024-11-01to2025-10-31_C000012167Member
Tag not found:  dei:SecurityExchangeName From2024-11-01to2025-10-31_C000012167Member
Processing: Emerging Markets Government Bond Index Fund
Extracting context:  From2024-11-01to2025-10-31_C000126408Member
Processing: Emerging Markets Government Bond Index Fund
Extracting context:  From2024-11-01to2025-10-31_C000126407Member
Tag not found:  dei:SecurityExchangeName From2024-11-01to2025-10-31_C000126407Member
Processing: Emerging Markets Government Bond Index Fund
Extracting context:  From2024-11-01to2025-10-31_C000126409Member
Tag not found:  dei:SecurityExchangeName From2024-11-01to2025-10-31_C000126409Member
Processing: Global Minimum Volatility Fund
Extracting context:  From2

In [22]:
from src.simple_rag.models.fund import FinancialHighlights
import pandas as pd
from IPython.display import display

print(len(df_performance))

total_df = pd.concat([df_performance[0], df_performance[1]], ignore_index=True)

returns_lookup = total_df.copy()

# Optional: Clean the total_return column (remove % sign if needed)
print(returns_lookup.head())
returns_lookup['fund_name'] = (
    returns_lookup['fund_name']
    .str.replace('\n', ' ', regex=False)
)
display(returns_lookup['fund_name'].unique())

numeric_columns = ['portfolio_turnover', 'expense_ratio', 'net_assets', 
                   'nav_beginning', 'nav_end', 'net_income_ratio', 'distribution_shares']


for col in numeric_columns:
    if col in returns_lookup.columns:
        if returns_lookup[col] is not None:  
            try:      
                returns_lookup[f'{col}_clean'] = (
                    returns_lookup[col]
                    .astype(str)
                    .str.replace('%', '')
                    .str.replace('$', '')
                    .str.replace(',', '')
                    .replace('N/A', '0')
                    .replace('', '0')
                    .replace('None', '0')
                    .astype(float)
                )
            except Exception as e:
                print(f"Error cleaning column '{col}': {str(e)}")
                print(returns_lookup[col].to_string())
count = 0
# Now you can efficiently match and update your funds
for fund_obj in funds_total:
    if fund_obj.ticker not in whitehall_funds:
        continue
    print(f"\nProcessing fund object: {fund_obj.name} - {fund_obj.share_class}")
    
    # Initialize annual returns
    if not hasattr(fund_obj, 'annual_returns') or fund_obj.annual_returns is None:
        fund_obj.annual_returns = {}
    
    # Clean the name: remove "Vanguard" and strip whitespace
    name = fund_obj.name.replace("Vanguard", "").strip()
    print(f"Cleaned name: '{name}'")
    
    if "™" in name:
        name = name.replace("™", "")
    elif "®" in name:
        name = name.replace("®", "")
    elif "\n" in name:
        name = name.replace("\n", " ")
        print(name)
    # Find matching rows based on fund name
    name_matches = returns_lookup[returns_lookup['fund_name'].str.strip().str.lower() == name.lower()]
    if len(name_matches) == 0:
        print("  No name matches found for ticker: ", fund_obj.ticker)
        continue
    
    print(f"  Found {len(name_matches)} name matches")
    
    # Clean share class (remove trademark symbol)
    share_class = fund_obj.share_class
    if "™" in share_class:
        share_class = share_class.replace("™", "")
    
    # Now match share class
    share_class_matches = name_matches[
        name_matches['share_class'].str.contains(share_class, case=False, na=False, regex=False)]
    
    if len(share_class_matches) == 0:
        print(f"  No share class matches found for '{share_class}' ticker: ", fund_obj.ticker)
        print(f"  Found {len(name_matches)} name records")
        count += 1
        # Add all matching returns
        for _, row in name_matches.iterrows():
            year = str(row['year'])
            highlights = FinancialHighlights(
                turnover=row.get('portfolio_turnover_clean', 0),
                expense_ratio=row.get('expense_ratio_clean', 0),
                total_return=row['total_return'],
                net_assets=row.get('net_assets_clean', 0),
                net_assets_value_begining=row.get('nav_beginning_clean', 0),
                net_assets_value_end=row.get('nav_end_clean', 0),
                net_income_ratio=row.get('net_income_ratio_clean', 0.0)
            )
            
            fund_obj.financial_highlights[year] = highlights
            print(f"  {year}: Total Return = {highlights.total_return}%, Expense Ratio = {highlights.expense_ratio}%, Net Assets = {highlights.net_assets}, Net Income Ratio = {highlights.net_income_ratio}, Turnover = {highlights.turnover}, Net Assets Value Begining = {highlights.net_assets_value_begining}, Net Assets Value End = {highlights.net_assets_value_end}")
        continue
    elif len(share_class_matches) > 5:
        print("  More than 5 share class matches found:")
        print(share_class_matches)
    
    
    print(f"  Found {len(share_class_matches)} matching records")
    count += 1
    # Add all matching returns
    for _, row in share_class_matches.iterrows():
        year = str(row['year'])
        highlights = FinancialHighlights(
            turnover=row.get('portfolio_turnover_clean', 0),
            expense_ratio=row.get('expense_ratio_clean', 0),
            total_return=row['total_return'],
            net_assets=row.get('net_assets_clean', 0),
            net_assets_value_begining=row.get('nav_beginning_clean', 0),
            net_assets_value_end=row.get('nav_end_clean', 0),
            net_income_ratio=row.get('net_income_ratio_clean', 0.0)
        )
        
        fund_obj.financial_highlights[year] = highlights
        print(f"  {year}: Total Return = {highlights.total_return}%, Expense Ratio = {highlights.expense_ratio}%, Net Assets = {highlights.net_assets}, Net Income Ratio = {highlights.net_income_ratio}, Turnover = {highlights.turnover}, Net Assets Value Begining = {highlights.net_assets_value_begining}, Net Assets Value End = {highlights.net_assets_value_end}")
print("count: ",count)
print("Total funds: ",whitehall_funds)
    

2
             fund_name share_class  year  net_assets  nav_beginning  nav_end  \
0  Mid-Cap Growth Fund        None  2025      3116.0          26.21    29.55   
1  Mid-Cap Growth Fund        None  2024      3042.0          19.38    26.21   
2  Mid-Cap Growth Fund        None  2023      2530.0          19.24    19.38   
3  Mid-Cap Growth Fund        None  2022      2956.0          38.72    19.24   
4  Mid-Cap Growth Fund        None  2021      5290.0          29.89    38.72   

   total_return  expense_ratio  net_income_ratio  portfolio_turnover  \
0         14.77           0.32              0.26              1285.0   
1         35.77           0.33              0.37                69.0   
2          0.99           0.37              0.37                87.0   
3        -32.22           0.35              0.14                71.0   
4         37.68           0.33             -0.04                98.0   

  distribution_shares  
0                None  
1                None  
2           

array(['Mid-Cap Growth Fund', 'Selected Value Fund',
       'Emerging Markets Government Bond Index Fund',
       'Global Minimum Volatility Fund',
       'International Dividend Appreciation Index Fund',
       'International High Dividend Yield Index Fund',
       'International Dividend Growth Fund',
       'Advice Select International Growth Fund',
       'Advice Select Dividend Growth Fund',
       'Advice Select Global Value Fund', 'International Explorer Fund',
       'High Dividend Yield Index Fund'], dtype=object)


Processing fund object: Mid-Cap Growth Fund - ShareClassType.INVESTOR
Cleaned name: 'Mid-Cap Growth Fund'
  Found 5 name matches
  No share class matches found for 'ShareClassType.INVESTOR' ticker:  VMGRX
  Found 5 name records
  2025: Total Return = 14.77%, Expense Ratio = 0.32%, Net Assets = 3116.0, Net Income Ratio = 0.26, Turnover = 1285.0, Net Assets Value Begining = 26.21, Net Assets Value End = 29.55
  2024: Total Return = 35.77%, Expense Ratio = 0.33%, Net Assets = 3042.0, Net Income Ratio = 0.37, Turnover = 69.0, Net Assets Value Begining = 19.38, Net Assets Value End = 26.21
  2023: Total Return = 0.99%, Expense Ratio = 0.37%, Net Assets = 2530.0, Net Income Ratio = 0.37, Turnover = 87.0, Net Assets Value Begining = 19.24, Net Assets Value End = 19.38
  2022: Total Return = -32.22%, Expense Ratio = 0.35%, Net Assets = 2956.0, Net Income Ratio = 0.14, Turnover = 71.0, Net Assets Value Begining = 38.72, Net Assets Value End = 19.24
  2021: Total Return = 37.68%, Expense Ratio 

In [23]:
import sys
%reload_ext autoreload
sys.path.append('../src')


from simple_rag.extraction.parser import compute_annual_returns

for fund in funds_total:
    if fund.ticker in performance_funds:
        returns = compute_annual_returns(fund.performance_table)
        print("\nFinal Annual Returns:")
        fund.annual_returns = returns
        print(f"  {fund.ticker}: {returns}")
        for year, return_ in returns.items():
            print(fund.financial_highlights.keys())
            if year not in fund.financial_highlights.keys():
                new_highlight = FinancialHighlights(
                year=int(year),
                total_return=return_,
                turnover=0.0,
                expense_ratio=0.0,
                net_assets=0.0,
                net_assets_value_begining=0.0,
                net_assets_value_end=0.0,
                net_income_ratio=0.0
                )
                fund.financial_highlights[year] = new_highlight
                print(f"    {year}: {new_highlight}")

Detected format: Year (YYYY)
Found years: [np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024), np.int64(2025)]
  2016 Return: $10,000.00 -> $9,451.00 = -5.49%
  2017 Return: $9,451.00 -> $11,595.00 = 22.69%
  2018 Return: $11,595.00 -> $12,710.00 = 9.62%
  2019 Return: $12,710.00 -> $14,434.00 = 13.56%
  2020 Return: $14,434.00 -> $17,079.00 = 18.32%
  2021 Return: $17,079.00 -> $23,515.00 = 37.68%
  2022 Return: $23,515.00 -> $15,939.00 = -32.22%
  2023 Return: $15,939.00 -> $16,097.00 = 0.99%
  2024 Return: $16,097.00 -> $21,856.00 = 35.78%
  2025 Return: $21,856.00 -> $25,084.00 = 14.77%

Final Annual Returns:
  VMGRX: {'2016': -5.49, '2017': 22.69, '2018': 9.62, '2019': 13.56, '2020': 18.32, '2021': 37.68, '2022': -32.22, '2023': 0.99, '2024': 35.78, '2025': 14.77}
dict_keys(['2025', '2024', '2023', '2022', '2021'])
    2016: turnover=0.0 expense_ratio=0.0 total_return=-5.49 

/home/luis/Desktop/code/RAG/notebooks/../src/simple_rag/extraction/parser.py:69: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['parsed_date'] = pd.to_datetime(df[date_col], errors='coerce')
/home/luis/Desktop/code/RAG/notebooks/../src/simple_rag/extraction/parser.py:69: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['parsed_date'] = pd.to_datetime(df[date_col], errors='coerce')
/home/luis/Desktop/code/RAG/notebooks/../src/simple_rag/extraction/parser.py:69: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['parsed_date'] = pd.to_datetime(df[date_col], errors='coerce')
/

In [24]:
import pickle
from pathlib import Path
import sys

# Add RAG directory to path
RAG_DIR = Path("/home/alvar/CascadeProjects/windsurf-project/RAG")
if str(RAG_DIR) not in sys.path:
    sys.path.insert(0, str(RAG_DIR))

# Define pickle file path
PKL_PATH = Path("./funds_backup_metadata.pkl")

print("Current working directory:", Path.cwd())
print("PKL_PATH resolves to:", PKL_PATH.resolve())

# Save to pickle file
try:
    with PKL_PATH.open("wb") as f:
        pickle.dump(funds_total, f)
    
    print(f"Successfully saved {len(funds_total)} funds to pickle file")
    print(f"File size: {PKL_PATH.stat().st_size / 1024:.2f} KB")
    
except Exception as e:
    print(f"Error saving to pickle file: {e}")

Current working directory: /home/luis/Desktop/code/RAG/notebooks
PKL_PATH resolves to: /home/luis/Desktop/code/RAG/notebooks/funds_backup_metadata.pkl
Successfully saved 125 funds to pickle file
File size: 542.00 KB


In [ ]:
import pickle
from pathlib import Path
from dataclasses import is_dataclass, asdict
import pandas as pd
import sys
from pathlib import Path
RAG_DIR = Path("/home/alvar/CascadeProjects/windsurf-project/RAG/src")
if str(RAG_DIR) not in sys.path:
    sys.path.insert(0, str(RAG_DIR))


PKL_PATH = Path("./funds_backup.pkl")
print("Current working directory:", Path.cwd())
print("PKL_PATH resolves to:", PKL_PATH.resolve())
with PKL_PATH.open("rb") as f:
    funds_total = pickle.load(f)

print(f"Loaded {len(funds_total)} funds from pickle file")

Current working directory: /home/alvar/CascadeProjects/windsurf-project/RAG/notebooks
PKL_PATH resolves to: /home/alvar/CascadeProjects/windsurf-project/RAG/notebooks/funds_backup.pkl
Loaded 125 funds from pickle file


## Ishares

### Low RAM mode

In [27]:
import gc
from concurrent.futures import ProcessPoolExecutor, as_completed
import signal

from concurrent.futures import ProcessPoolExecutor, as_completed
import resource
import os
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor, as_completed
import pandas as pd
from typing import List
import sys
from pathlib import Path
from tqdm import tqdm
%reload_ext autoreload
from src.simple_rag.extraction.parser import BlackRockFiling
from edgar import set_identity, Company
from src.simple_rag.models.fund import FilingMetadata
set_identity("luis.alvarez.conde@alumnos.upm.es")

class TimeoutException(Exception):
    pass

def timeout_handler(signum, frame):
    raise TimeoutException("HTML parsing timed out")

# --- 1. UPDATED WORKER (Remove the memory limit code) ---
def process_single_filing_multiprocess(filing_data):
    """
    Process a single filing. 
    REMOVED: resource.setrlimit (Let the OS manage memory)
    """
    try:
        # Unpack data
        html_content, report_date, accession_number, filing_date, form, url = filing_data
        
        # Import inside function
        from src.simple_rag.extraction.parser import BlackRockFiling
        from src.simple_rag.models.fund import FilingMetadata
        
        parser = BlackRockFiling(html_content)
        funds = parser.get_funds()
        
        performance_tickers = []
        df_performance = None        
        
        filing_metadata = FilingMetadata(
            accession_number=accession_number,
            reporting_date=report_date,
            filing_date=filing_date,
            form=form,
            url=url
        )
        
        all_tickers = []
        processed_funds = []
        
        for fund in funds:
            fund.ncsr_metadata = filing_metadata
            processed_funds.append(fund)
            all_tickers.append(fund.ticker)
            
            if fund.performance_table is not None:
                if fund.ticker not in performance_tickers:
                    performance_tickers.append(fund.ticker)
        
        try:
            df_performance = parser.get_financial_highlights2()
        except Exception as e:
            pass # Suppress minor parsing errors
        
        return {
            'funds': processed_funds,
            'all_tickers': all_tickers,
            'performance_tickers': performance_tickers,
            'df_performance': df_performance,
            'report_date': report_date
        }

    except Exception as e:
        print(f"❌ Error in worker: {e}")
        return None

# --- 2. NEW BATCHED MAIN LOOP ---

# Configuration
BATCH_SIZE = 5  # Process only 5 filings at a time to save RAM
MAX_WORKERS = 3 # Keep this low (2-4) to prevent CPU/RAM saturation


ticker = "HEZU"
fund = Company(ticker)
all_filings = fund.get_filings(form="N-CSR")

if all_filings:
    unique_dates = sorted({f.report_date for f in all_filings if f.report_date})
    print("Unique report dates:", unique_dates)
    
    # Filter for filings from 2024-08-31 onward
    cutoff_date = "2024-09-31"
    latest_filings = [
        f for f in all_filings 
        if f.report_date and f.report_date >= cutoff_date
    ]
    
    print("Found filings: ", len(latest_filings), "from", cutoff_date, "onward")
    
    # Optional: Show the dates of filtered filings
    print("Filtered filing dates:", sorted({f.report_date for f in latest_filings}))

# Global results containers
all_funds_total = []

performance_funds = []
df_performances = []
ishares_funds = []

# Helper to chunk the list
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

print(f"🚀 Starting processing with Batch Size: {BATCH_SIZE} | Workers: {MAX_WORKERS}")

# Iterate through filings in small groups
for batch_index, filing_batch in enumerate(chunker(latest_filings, BATCH_SIZE)):
    
    print(f"\n📦 Processing Batch {batch_index + 1} ({len(filing_batch)} filings)...")
    
    # 1. Prepare Data for this Batch ONLY
    batch_data = []
    
    for filing in filing_batch:
        try:
            # Your existing timeout logic
            signal.signal(signal.SIGALRM, timeout_handler)
            signal.alarm(10) # 10s timeout
            html_content = filing.html()
            signal.alarm(0)
            
            if html_content:
                batch_data.append((
                    html_content, 
                    filing.report_date,
                    filing.accession_number,
                    filing.filing_date,
                    filing.form,
                    filing.url
                ))
        except Exception as e:
            signal.alarm(0)
            print(f"   ⚠️ Skipped {filing.report_date}: {e}")

    # 2. Process this Batch Immediately
    if batch_data:
        with ProcessPoolExecutor(max_workers=MAX_WORKERS) as executor:
            future_to_data = {executor.submit(process_single_filing_multiprocess, d): d for d in batch_data}
            
            for future in as_completed(future_to_data):
                res = future.result()
                if res:
                    # Aggregate results
                    funds_total.extend(res['funds'])
                    ishares_funds.extend(res['all_tickers'])
                    performance_funds.extend(res['performance_tickers'])
                    if res['df_performance'] is not None:
                        df_performances.append(res['df_performance'])
    
    # 3. CRITICAL: Clear memory
    print(f"   🧹 Cleaning up batch memory...")
    del batch_data
    gc.collect() # Force Python to release RAM

print(f"\n✅ DONE!")
print(f"Total funds: {len(funds_total)}")
print(f"Performance Tables: {len(df_performances)}")

Unique report dates: ['2003-04-30', '2003-07-31', '2004-02-29', '2004-03-31', '2004-04-30', '2004-07-31', '2005-02-28', '2005-03-31', '2005-04-30', '2005-07-31', '2006-02-28', '2006-03-31', '2006-04-30', '2006-07-31', '2007-02-28', '2007-03-31', '2007-04-30', '2007-07-31', '2008-02-29', '2008-03-31', '2008-04-30', '2008-07-31', '2009-02-28', '2009-03-31', '2009-04-30', '2009-07-31', '2009-08-31', '2010-02-28', '2010-03-31', '2010-04-30', '2010-07-31', '2010-08-31', '2011-02-28', '2011-03-31', '2011-04-30', '2011-07-31', '2011-08-31', '2011-10-31', '2012-02-29', '2012-03-31', '2012-04-30', '2012-07-31', '2012-08-31', '2012-10-31', '2013-02-28', '2013-03-31', '2013-04-30', '2013-07-31', '2013-08-31', '2013-10-31', '2014-02-28', '2014-03-31', '2014-04-30', '2014-07-31', '2014-08-31', '2014-10-31', '2015-02-28', '2015-03-31', '2015-04-30', '2015-07-31', '2015-08-31', '2015-10-31', '2016-02-29', '2016-03-31', '2016-04-30', '2016-07-31', '2016-08-31', '2016-10-31', '2017-02-28', '2017-03-31'

In [30]:
for fund in funds_total:
    print(fund.name)
    print(fund.ncsr_metadata)
    break

Vanguard Extended Market Index Fund
accession_number='0001104659-25-020311' filing_date=datetime.date(2025, 3, 4) reporting_date=datetime.date(2024, 12, 31) url='https://www.sec.gov/Archives/edgar/data/36405/0001104659-25-020311-index.html' form='N-CSR'


In [1]:
from concurrent.futures import ProcessPoolExecutor, as_completed
import resource
import os
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor, as_completed
import pandas as pd
from typing import List
import sys
from pathlib import Path
from tqdm import tqdm
%reload_ext autoreload
from src.simple_rag.extraction.parser import BlackRockFiling
from edgar import set_identity, Company
from src.simple_rag.models.fund import FilingMetadata
set_identity("luis.alvarez.conde@alumnos.upm.es")

def process_single_filing_multiprocess(filing_data):
    """Process a single filing with memory limits"""
    try:
        # Limit memory to 1.5GB per process
        soft, hard = resource.getrlimit(resource.RLIMIT_AS)
        resource.setrlimit(resource.RLIMIT_AS, (1500 * 1024 * 1024, hard))
        
        # Your existing code here...
        html_content, report_date, accession_number, filing_date, form, url = filing_data
        
        import sys
        from pathlib import Path
        sys.path.append('../src')
        from src.simple_rag.extraction.parser import BlackRockFiling
        from src.simple_rag.models.fund import FilingMetadata
        
        parser = BlackRockFiling(html_content)
        funds = parser.get_funds()
        
        performance_funds = []
        df_performance = None        
        count = 0
        
        filing_metadata = FilingMetadata(
            accession_number=accession_number,
            reporting_date=report_date,
            filing_date=filing_date,
            form=form,
            url=url
        )
        
        all_tickers = []  # Fix: collect all tickers
        
        for fund in funds:
            if fund.performance_table is not None:
                if fund.ticker not in performance_funds:
                    performance_funds.append(fund.ticker)
                    count += 1
            
            all_tickers.append(fund.ticker)  # Fix: collect here instead
            fund.ncsr_metadata = filing_metadata
        
        print(f"Calling get_financial_highlights2 for {report_date}")
        df_performance = parser.get_financial_highlights2()
        
        print(f"Filing {report_date}: Found {count} funds with performance tables, Total funds: {len(funds)}")
        
        return {
            'funds': funds,
            'all_tickers': all_tickers,  # Fix: return all tickers
            'performance_tickers': performance_funds,
            'df_performance': df_performance,
            'report_date': report_date
        }
    except MemoryError:
        print(f"❌ Memory limit exceeded for filing {filing_data[1]}")
        return None
    except Exception as e:
        print(f"Error processing filing: {e}")
        return None

# Calculate safe worker count based on available RAM
import psutil
available_ram_gb = psutil.virtual_memory().available / (1024**3)
# Assume each worker needs ~2GB, leave 2GB for system
max_workers = max(1, min(4, int((available_ram_gb - 2) / 2)))
max_workers = 1
print(f"Using {max_workers} workers based on {available_ram_gb:.1f}GB available RAM")

ticker = "HEZU"
fund = Company(ticker)
all_filings = fund.get_filings(form="N-CSR")

if all_filings:
    unique_dates = sorted({f.report_date for f in all_filings if f.report_date})
    print("Unique report dates:", unique_dates)
    
    # Filter for filings from 2024-08-31 onward
    cutoff_date = "2024-09-31"
    latest_filings = [
        f for f in all_filings 
        if f.report_date and f.report_date >= cutoff_date
    ]
    
    print("Found filings: ", len(latest_filings), "from", cutoff_date, "onward")
    
    # Optional: Show the dates of filtered filings
    print("Filtered filing dates:", sorted({f.report_date for f in latest_filings}))


# Prepare data for multiprocessing (fetch HTML first)
filing_data_list = []
failed_filings = []
import signal

class TimeoutException(Exception):
    pass

def timeout_handler(signum, frame):
    raise TimeoutException("HTML parsing timed out")

sgml_filings = []

for filing in latest_filings:
    try:
        # Set a 10-second timeout for HTML fetching
        signal.signal(signal.SIGALRM, timeout_handler)
        signal.alarm(10)
        
        html_content = filing.html()
        
        # Cancel the alarm
        signal.alarm(0)
        
        if html_content:  # Only add if HTML content exists
            filing_data_list.append((
                html_content, 
                filing.report_date,
                filing.accession_number,
                filing.filing_date,
                filing.form,
                filing.url
            ))
        else:
            print(f"⚠️  No HTML content for filing: {filing.report_date}")
            failed_filings.append(filing)
            
    except TimeoutException:
        signal.alarm(0)
        print(f"⏭️  Timeout (likely SGML): {filing.report_date}")
        sgml_filings.append(filing)
        
    except ValueError as e:
        signal.alarm(0)
        print(f"❌ Error processing filing {filing.report_date}: {e}")
        failed_filings.append(filing)
        
    except Exception as e:
        signal.alarm(0)
        print(f"❌ Unexpected error for filing {filing.report_date}: {e}")
        failed_filings.append(filing)

print(f"✅ Successfully prepared {len(filing_data_list)} filings")
print(f"⏭️  Skipped {len(sgml_filings)} SGML/timeout filings")
print(f"❌ Failed to prepare {len(failed_filings)} filings")
# Continue with successful filings only

performance_funds = []
df_performances = []
funds_total = []  # Make sure this is defined

# Process with limited workers
with ProcessPoolExecutor(max_workers=max_workers) as executor:
    future_to_data = {executor.submit(process_single_filing_multiprocess, data): data 
                      for data in filing_data_list}
    
    for future in tqdm(as_completed(future_to_data), total=len(filing_data_list), desc="Processing filings"):
        result = future.result()
        if result:
            funds_total.extend(result['funds'])
            ishares_funds.extend(result['all_tickers'])  # Fix: use returned tickers
            performance_funds.extend(result['performance_tickers'])
            
            if result['df_performance'] is not None:
                df_performances.append(result['df_performance'])

print(f"\nTotal funds processed: {len(funds_total)}")
print(f"Unique iShares funds: {len(set(ishares_funds))}")
print(f"Performance dataframes: {len(df_performances)}")

/home/luis/Desktop/code/RAG/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using 1 workers based on 5.9GB available RAM


KeyboardInterrupt: 

In [ ]:
from concurrent.futures import ProcessPoolExecutor, as_completed
import pandas as pd
from typing import List
import sys
from pathlib import Path
from tqdm import tqdm
%reload_ext autoreload
from src.simple_rag.extraction.parser import BlackRockFiling
from edgar import set_identity, Company
from src.simple_rag.models.fund import FilingMetadata
set_identity("luis.alvarez.conde@alumnos.upm.es")

ticker = "HEZU"
fund = Company(ticker)
all_filings = fund.get_filings(form="N-CSR")

def process_single_filing_multiprocess(filing_data):
    """
    Process a single filing (for multiprocessing).
    Note: Must pass serializable data, not the filing object directly
    """
    try:
        # Import inside function for multiprocessing
        import sys
        from pathlib import Path
        sys.path.append('../src')
        from src.simple_rag.extraction.parser import BlackRockFiling
        
        html_content, report_date, accession_number, filing_date, form, url = filing_data
        parser = BlackRockFiling(html_content)
        funds = parser.get_funds()
        
        performance_funds = []
        df_performance = None        
        count = 0
        filing_metadata = FilingMetadata(
        accession_number=accession_number,
        reporting_date=report_date,
        filing_date=filing_date,
        form=form,
        url=url

    )
        for fund in funds:

            if fund.performance_table is not None:
                if fund.ticker not in performance_funds:
                    performance_funds.append(fund.ticker)
                    count += 1
            if fund.ticker not in ishares_funds:
                ishares_funds.append(fund.ticker)
            fund.ncsr_metadata = filing_metadata
            print("Calling get_financial_highlights2")
        df_performance = parser.get_financial_highlights2()
        
        print(f"Filing {report_date}: Found {count} funds with performance tables, Total funds: {len(funds)}")
        
        return {
            'funds': funds,
            'performance_tickers': performance_funds,
            'df_performance': df_performance,
            'report_date': report_date
        }
    except Exception as e:
        print(f"Error processing filing: {e}")
        return None

if all_filings:
    unique_dates = sorted({f.report_date for f in all_filings if f.report_date})
    print("Unique report dates:", unique_dates)
    
    # Filter for filings from 2024-08-31 onward
    cutoff_date = "2024-09-31"
    latest_filings = [
        f for f in all_filings 
        if f.report_date and f.report_date >= cutoff_date
    ]
    
    print("Found filings: ", len(latest_filings), "from", cutoff_date, "onward")
    
    # Optional: Show the dates of filtered filings
    print("Filtered filing dates:", sorted({f.report_date for f in latest_filings}))

# Prepare data for multiprocessing (fetch HTML first)
filing_data_list = []
failed_filings = []
import signal

class TimeoutException(Exception):
    pass

def timeout_handler(signum, frame):
    raise TimeoutException("HTML parsing timed out")

sgml_filings = []

for filing in latest_filings:
    try:
        # Set a 10-second timeout for HTML fetching
        signal.signal(signal.SIGALRM, timeout_handler)
        signal.alarm(10)
        
        html_content = filing.html()
        
        # Cancel the alarm
        signal.alarm(0)
        
        if html_content:  # Only add if HTML content exists
            filing_data_list.append((
                html_content, 
                filing.report_date,
                filing.accession_number,
                filing.filing_date,
                filing.form,
                filing.url
            ))
        else:
            print(f"⚠️  No HTML content for filing: {filing.report_date}")
            failed_filings.append(filing)
            
    except TimeoutException:
        signal.alarm(0)
        print(f"⏭️  Timeout (likely SGML): {filing.report_date}")
        sgml_filings.append(filing)
        
    except ValueError as e:
        signal.alarm(0)
        print(f"❌ Error processing filing {filing.report_date}: {e}")
        failed_filings.append(filing)
        
    except Exception as e:
        signal.alarm(0)
        print(f"❌ Unexpected error for filing {filing.report_date}: {e}")
        failed_filings.append(filing)

print(f"✅ Successfully prepared {len(filing_data_list)} filings")
print(f"⏭️  Skipped {len(sgml_filings)} SGML/timeout filings")
print(f"❌ Failed to prepare {len(failed_filings)} filings")
# Continue with successful filings only

performance_funds = []
df_performances = []
ishares_funds = []

# Use ProcessPoolExecutor
with ProcessPoolExecutor(max_workers=2) as executor:
    future_to_data = {executor.submit(process_single_filing_multiprocess, data): data 
                      for data in filing_data_list}
    
    for future in tqdm(as_completed(future_to_data), total=len(filing_data_list), desc="Processing filings"):
        result = future.result()
        if result:
            ishares_funds.extend(result['funds'])
            funds_total.extend(result['funds'])
            performance_funds.extend(result['performance_tickers'])
            
            if result['df_performance'] is not None:
                df_performances.append(result['df_performance'])

print(len(df_performances))
print(f"Total funds processed: {len(ishares_funds)}")


Unique report dates: ['2003-04-30', '2003-07-31', '2004-02-29', '2004-03-31', '2004-04-30', '2004-07-31', '2005-02-28', '2005-03-31', '2005-04-30', '2005-07-31', '2006-02-28', '2006-03-31', '2006-04-30', '2006-07-31', '2007-02-28', '2007-03-31', '2007-04-30', '2007-07-31', '2008-02-29', '2008-03-31', '2008-04-30', '2008-07-31', '2009-02-28', '2009-03-31', '2009-04-30', '2009-07-31', '2009-08-31', '2010-02-28', '2010-03-31', '2010-04-30', '2010-07-31', '2010-08-31', '2011-02-28', '2011-03-31', '2011-04-30', '2011-07-31', '2011-08-31', '2011-10-31', '2012-02-29', '2012-03-31', '2012-04-30', '2012-07-31', '2012-08-31', '2012-10-31', '2013-02-28', '2013-03-31', '2013-04-30', '2013-07-31', '2013-08-31', '2013-10-31', '2014-02-28', '2014-03-31', '2014-04-30', '2014-07-31', '2014-08-31', '2014-10-31', '2015-02-28', '2015-03-31', '2015-04-30', '2015-07-31', '2015-08-31', '2015-10-31', '2016-02-29', '2016-03-31', '2016-04-30', '2016-07-31', '2016-08-31', '2016-10-31', '2017-02-28', '2017-03-31'

Processing filings:   0%|          | 0/22 [00:00<?, ?it/s]

Processing: iShares iBonds 1-5 Year Corporate Ladder ETF
Extracting context:  From2024-11-01to2025-10-31_C000254885Member
Tag not found:  oef:ClassName From2024-11-01to2025-10-31_C000254885Member
Unknown Table:        Footnote                   Description
0  Footnote(a)  Excludes money market funds.
Unknown table type:       Footnote                   Description
0  Footnote(a)  Excludes money market funds.
Processing: iShares iBonds 1-5 Year High Yield and Income Ladder ETF
Extracting context:  From2024-11-01to2025-10-31_C000254886Member
Tag not found:  oef:ClassName From2024-11-01to2025-10-31_C000254886Member
Unknown Table:        Footnote                   Description
0  Footnote(a)  Excludes money market funds.
Unknown table type:       Footnote                   Description
0  Footnote(a)  Excludes money market funds.
Processing: iShares iBonds 1-5 Year TIPS Ladder ETF
Extracting context:  From2024-11-01to2025-10-31_C000254887Member
Tag not found:  oef:ClassName From2024-11-01to2

Processing filings:   5%|▍         | 1/22 [00:01<00:39,  1.90s/it]

Processing: iShares Climate Conscious & Transition MSCI USA ETF
Extracting context:  From2024-09-01to2025-08-31_C000242847Member
Tag not found:  oef:ClassName From2024-09-01to2025-08-31_C000242847Member
Processing: iShares ESG Advanced MSCI USA ETF
Extracting context:  From2024-09-01to2025-08-31_C000219749Member
Tag not found:  oef:ClassName From2024-09-01to2025-08-31_C000219749Member
Processing: iShares ESG Aware MSCI USA ETF
Extracting context:  From2024-09-01to2025-08-31_C000174221Member
Tag not found:  oef:ClassName From2024-09-01to2025-08-31_C000174221Member
Processing: iShares ESG Aware MSCI USA Growth ETF
Extracting context:  From2024-09-01to2025-08-31_C000240124Member
Tag not found:  oef:ClassName From2024-09-01to2025-08-31_C000240124Member
Processing: iShares ESG Aware MSCI USA Small-Cap ETF
Extracting context:  From2024-09-01to2025-08-31_C000199548Member
Tag not found:  oef:ClassName From2024-09-01to2025-08-31_C000199548Member
Processing: iShares ESG Aware MSCI USA Value ETF


...

Calling get_financial_highlights2
Calling get_financial_highlights2
Calling get_financial_highlights2
Calling get_financial_highlights2
Calling get_financial_highlights2
Calling get_financial_highlights2
Calling get_financial_highlights2
Calling get_financial_highlights2
Calling get_financial_highlights2
Calling get_financial_highlights2
Calling get_financial_highlights2
Calling get_financial_highlights2
Calling get_financial_highlights2
Calling get_financial_highlights2
Calling get_financial_highlights2
Calling get_financial_highlights2
Calling get_financial_highlights2
Calling get_financial_highlights2
Calling get_financial_highlights2
Calling get_financial_highlights2
Calling get_financial_highlights2
Found 3 potential Financial Highlights sections
Total funds extracted: 0
Filing 2025-03-31: Found 18 funds with performance tables, Total funds: 21


Processing filings:  77%|███████▋  | 17/22 [03:04<00:55, 11.14s/it]

No data obtained
Processing: iShares iBonds Dec 2054 Term Treasury ETF
Extracting context:  FY2024_C000250199Member
Tag not found:  oef:FactorsAffectingPerfTextBlock FY2024_C000250199Member
Unknown Table:        0                                                  1
0  ​(a)                       Excludes money market funds.
1    ​*  Credit quality ratings shown reflect the ratin...
Unknown table type:       0                                                  1
0  ​(a)                       Excludes money market funds.
1    ​*  Credit quality ratings shown reflect the ratin...
No data obtained
Processing: iShares 0-3 Month Treasury Bond ETF
Extracting context:  From2024-03-01to2025-02-28_C000219740Member
Tag not found:  oef:ClassName From2024-03-01to2025-02-28_C000219740Member
No data obtained
Processing: iShares iBonds Oct 2034 Term TIPS ETF
Extracting context:  FY2024_C000249962Member
Unknown Table:        Footnote                   Description
0  Footnote(a)  Excludes money market funds

Processing filings:  82%|████████▏ | 18/22 [03:13<00:41, 10.37s/it]

Tag not found:  dei:SecurityExchangeName FY2024_C000249962Member
Tag not found:  oef:FactorsAffectingPerfTextBlock FY2024_C000249962Member
Unknown Table:        0                                                  1
0  ​(a)                       Excludes money market funds.
1    ​*  Credit quality ratings shown reflect the ratin...
Unknown table type:       0                                                  1
0  ​(a)                       Excludes money market funds.
1    ​*  Credit quality ratings shown reflect the ratin...


In [41]:
for fund in ishares_funds:
    print(fund)
   

LDRC
LDRH
LDRI
LDRT
HEZU
HEWJ
DMXF
EMXF
ESGD
LDEM
EWZS
MCHI
TCHI
ECNS
EDEN
EQLT
EVLU
AQLT
SDG
INDA
SMIN
EIDO
EWJV
KWT
ENZL
ENOR
EPU
EPHE
EPOL
QAT
KSA
UAE
EWU
WSML
PABD
USCL
USXF
ESGU
EGUS
ESML
EVUS
SUSL
PABU
EWUS
EFNL
EIRL
SHYG
SLQD
ISHG
TLTW
QLTA
HYBB
USHY
CMBS
ICVT
ISTB
IAGG
HYXF
FALN
FLOT
GNMA
HYGW
IBHE
IBHF
IBHG
IBHK
IBDQ
IBDR
IBDS
IBDT
IBDU
IBDV
IBMS
IBDW
IBDZ
IBTP
IBGA
IBGK
IBIK
IGOV
LQDW
BEMB
EMB
IWMW
IVVW
SYSB
TFLO
BGRN
TWOX
MMAX
MAXJ
SMAX
DMAX
EAOK
EAOM
EAOR
EAOA
SMMV
LRGF
SMLF
TECB
AOK
AOM
AOR
AOA
IYLD
ESMV
USMV
MTUM
QUAL
SIZE
VLUE
IRTR
ITDB
ITDC
ITDD
ITDE
ITDF
ITDG
ITDH
ITDI
ITDJ
ETEC
FXI
IEFA
IEUR
IDEV
IPAC
IXUS
HAWX
HEFA
HSCZ
IHAK
IBAT
XT
IDAT
IDNA
GLOF
INTF
ISCF
ACWI
ACWX
CRBN
AAXJ
CNYA
EFA
EFG
EFAV
SCZ
EFV
EUFN
IEUS
IMTM
IQLT
IVLU
TOK
IBRN
IDRV
SHYG
SLQD
ISHG
TLTW
QLTA
HYBB
USHY
CMBS
ICVT
ISTB
IAGG
HYXF
FALN
FLOT
GNMA
HYGW
IBHD
IBHE
IBHF
IBHG
IBHK
IBDP
IBDQ
IBDR
IBDS
IBDT
IBDU
IBDV
IBMS
IBDW
IBDZ
IBTP
IBGA
IBGK
IBIK
IGOV
LQDW
BEMB
EMB
IWMW
IVVW
TFLO
FIBR
BGRN
ERET
ICLN
R

In [38]:
for fund in funds_total:
    print(fund)
    break

ncsr_metadata=FilingMetadata(accession_number='0001104659-25-020311', filing_date=datetime.date(2025, 3, 4), reporting_date=datetime.date(2024, 12, 31), url='https://www.sec.gov/Archives/edgar/data/36405/0001104659-25-020311-index.html', form='N-CSR') summary_prospectus_metadata=None nport_metadata=None name='Vanguard Extended Market Index Fund' registrant='Vanguard Index Funds' provider=None context_id='FY2024_C000007779Member' share_class=<ShareClassType.INVESTOR: 'Investor Shares'> ticker='VEXMX' security_exchange='N/A' series_id=None costs_per_10k=21 expense_ratio=0.19 net_assets=111156.0 turnover_rate=11.0 advisory_fees=1799.0 n_holdings=3485 report_date=datetime.date(2024, 12, 31) annual_returns={} performance={} avg_annual_returns=                                                   0       1        2  \
0                       Average Annual Total Returns     NaN      NaN   
1                                                NaN  1 Year  5 Years   
2                                

In [42]:
import pandas as pd
import re
%reload_ext autoreload
from simple_rag.models.fund import FinancialHighlights

if df_performances:
    df_performance = pd.concat(df_performances, ignore_index=True)
else:
    df_performance = pd.DataFrame() # Empty fallback
    print("No performance data found.")

print(df_performance.head())

def clean_financial_number(val):
    """
    Parses financial strings like '23.19 %(b)' or '(24.82 )%'.
    - Extracts the numerical value.
    - Handles (12.34) as negative -12.34.
    - Ignores footnote markers like (a), (b).
    - Removes %, $, and commas.
    """
    if pd.isna(val) or val is None:
        return None
    
    # Convert to string and strip whitespace
    s = str(val).strip()
    
    # 1. Regex to find the number (handles decimals and commas)
    # Looks for digits, optional commas, and optional decimal part
    match = re.search(r'(\d{1,3}(?:,\d{3})*\.?\d*|\d*\.?\d+)', s)
    
    if not match:
        return None
        
    # Get the raw number string (e.g., "24.82" or "1,234.56")
    num_str = match.group(0)
    
    # 2. Check for negative indication: "(" at the start of the string
    # Accounting format always puts the negative parenthesis at the start: (24.82)%
    is_negative = s.startswith('(')
    
    try:
        # Remove commas and convert to float
        clean_num = float(num_str.replace(',', ''))
        
        # Apply negative sign if detected
        return -clean_num if is_negative else clean_num
        
    except ValueError:
        return None
returns_lookup = df_performance.copy()


# Apply to all financial columns
financial_cols = ['total_return', 'expense_ratio', 'net_income_ratio', 'portfolio_turnover', 'nav_end', 'nav_beginning', 'net_assets']
for col in financial_cols:
    if col in returns_lookup.columns:
        returns_lookup[f'{col}_clean'] = returns_lookup[col].apply(clean_financial_number)

ishares_tickers = [fund_obj for fund_obj in ishares_funds]
print("Tickers in ishares_funds:", ishares_tickers)


# Now you can efficiently match and update your funds
for fund_obj in funds_total:
    
    if fund_obj.ticker not in ishares_tickers:
        continue

    print(f"\nProcessing fund object: {fund_obj.name} - {fund_obj.share_class}")
    # Initialize annual returns
    if not hasattr(fund_obj, 'annual_returns') or fund_obj.annual_returns is None:
        fund_obj.annual_returns = {}

    if not hasattr(fund_obj, 'financial_highlights') or fund_obj.financial_highlights is None:
        fund_obj.financial_highlights = {}
    
    # Clean the name: remove "Vanguard" and strip whitespace
    name = fund_obj.name.replace("Vanguard", "").strip()
    print(f"Cleaned name: '{name}'")
    
    if "®" in name:
        name = name.replace("®", "")
    if "™" in name:
        name = name.replace("™", "")
        
    # Find matching rows based on fund name
    name_matches = returns_lookup[returns_lookup['fund_name'].str.contains(name, case=False, na=False, regex=False)]
    
    if len(name_matches) == 0:
        print("  No name matches found")
        continue
    
    print(f"  Found {len(name_matches)} name matches")
    
    # Clean share class (remove trademark symbol)
    fund_obj.share_class = "ETF Shares"
    share_class = fund_obj.share_class
    
    if "™" in share_class:
        share_class = share_class.replace("™", "")
    
    # Now match share class
    share_class_matches = name_matches[
        name_matches['share_class'].str.contains(share_class, case=False, na=False, regex=False)]
    
    if name_matches['share_class'].isna().all():
        fund_obj.annual_returns = dict(zip(name_matches['year'], name_matches['total_return_clean']))
        print("Annual return: ", fund_obj.annual_returns)
        continue
        
    if len(share_class_matches) == 0:
        print(f"  No share class matches found for '{share_class}'")
        print(f"  Available share classes: {name_matches['share_class'].unique()}")
        continue
    
    print(f"  Found {len(share_class_matches)} matching records")
    
    # Add all matching returns
    for _, row in share_class_matches.iterrows():
        year = str(row['year'])
        
        # Store annual return
        fund_obj.annual_returns[year] = row['total_return_clean']
        
        # Store full financial highlights snapshot
        fund_obj.financial_highlights[year] = FinancialHighlights(
            turnover=row.get('portfolio_turnover_clean'),
            expense_ratio=row.get('expense_ratio_clean'),
            total_return=row.get('total_return_clean'),
            net_assets=row.get('net_assets'),  # You may need to add this cleaning
            net_assets_value_begining=row.get('nav_beginning_clean'),
            net_assets_value_end=row.get('nav_end_clean') ,
            net_income_ratio=row.get('net_income_ratio_clean')
        )
    
    print(f"  Annual returns: {fund_obj.annual_returns}")
    print(f"  Financial highlights years: {list(fund_obj.financial_highlights.keys())}")
    for key, value in fund_obj.financial_highlights.items():
        print(f"    {key}: {value}")

                            fund_name share_class  year   net_assets  \
0  iShares ESG Advanced MSCI EAFE ETF  ETF Shares  2025  821452000.0   
1  iShares ESG Advanced MSCI EAFE ETF  ETF Shares  2024  708068000.0   
2  iShares ESG Advanced MSCI EAFE ETF  ETF Shares  2023  475876000.0   
3  iShares ESG Advanced MSCI EAFE ETF  ETF Shares  2022  354432000.0   
4  iShares ESG Advanced MSCI EAFE ETF  ETF Shares  2021  245846000.0   

   nav_beginning  nav_end  total_return  expense_ratio  net_income_ratio  \
0          71.52    74.00          6.47           0.12              2.43   
1          59.48    71.52         23.19           0.12              2.39   
2          51.37    59.48         18.17           0.12              2.41   
3          70.24    51.37          0.00           0.12              2.73   
4          55.79    70.24         27.47           0.12              2.06   

   portfolio_turnover distribution_shares  
0                14.0                None  
1                24.0 

In [43]:
import re
from collections import defaultdict
import pandas as pd

def infer_first_col_format(value: object) -> str:
    if value is None or (isinstance(value, float) and pd.isna(value)):
        return "EMPTY"

    s = str(value).strip()
    if s == "" or s.lower() == "nan":
        return "EMPTY"

    # Jan 23, Aug 15
    if re.match(r"^[A-Za-z]{3}\s+\d{2}$", s):
        return "MON_YY"

    # 2015
    if re.match(r"^\d{4}$", s):
        return "YYYY"

    # 2024-08-31
    if re.match(r"^\d{4}-\d{2}-\d{2}$", s):
        return "YYYY_MM_DD"

    # 08/31/24 or 8/31/2024
    if re.match(r"^\d{1,2}/\d{1,2}/\d{2,4}$", s):
        return "MM_DD_YY(YY)"

    # 31/08/24 (if you ever have EU style)
    if re.match(r"^\d{1,2}-\d{1,2}-\d{2,4}$", s):
        return "DD_MM_YY(YY)_or_MM_DD_YY(YY)_DASH"

    # Fallbacks
    if re.search(r"\d", s):
        return "OTHER_HAS_DIGITS"

    return "OTHER_TEXT"


def describe_first_column_formats(
    dfs,
    names=None,
    samples_per_df=3,
    max_groups_to_show=50,
    max_dfs_per_group_to_print=5,
):
    if names is None:
        names = [f"df[{i}]" for i in range(len(dfs))]

    groups = defaultdict(list)

    for name, df in zip(names, dfs):
        if df is None or not isinstance(df, pd.DataFrame) or df.empty:
            groups["EMPTY_DF"].append((name, df))
            continue

        first_col = df.columns[0]
        # take first non-empty sample from first column
        series = df[first_col].astype(str)
        sample_vals = [v for v in series.head(20).tolist() if str(v).strip() and str(v).lower() != "nan"]

        fmt = infer_first_col_format(sample_vals[0]) if sample_vals else "EMPTY_FIRST_COL"
        groups[fmt].append((name, df))

    sorted_groups = sorted(groups.items(), key=lambda kv: len(kv[1]), reverse=True)

    print(f"Total dataframes: {len(dfs)}")
    print(f"Unique first-column formats: {len(sorted_groups)}\n")

    for gi, (fmt, members) in enumerate(sorted_groups[:max_groups_to_show], start=1):
        print("=" * 100)
        print(f"Group #{gi}: {fmt}")
        print(f"Count: {len(members)}")

        example_shapes = [m[1].shape for m in members if isinstance(m[1], pd.DataFrame)]
        print(f"Example shapes (first 10): {example_shapes[:10]}")

        # Print a few examples per group
        for ex_i, (name, df) in enumerate(members[:max_dfs_per_group_to_print], start=1):
            if df is None or not isinstance(df, pd.DataFrame) or df.empty:
                print(f"  [Example {ex_i}] {name}: EMPTY/None")
                continue

            first_col = df.columns[0]
            vals = [v for v in df[first_col].head(20).tolist() if str(v).strip() and str(v).lower() != "nan"]
            vals = vals[:samples_per_df]

            print(f"  [Example {ex_i}] {name}")
            print(f"    first_col: {first_col!r}")
            print(f"    columns: {list(df.columns)[:12]}{' ...' if len(df.columns) > 12 else ''}")
            print(f"    first_col_samples: {vals}")

        print()


# Example usage with your list of performance tables
performances = []
perf_names = []
for i, fund in enumerate(funds_total):
    if fund.ticker in performance_funds and fund.performance_table is not None:
        performances.append(fund.performance_table)
        perf_names.append(f"{fund.ticker} | {fund.name} | {fund.share_class}")

describe_first_column_formats(performances, names=perf_names)

Total dataframes: 105
Unique first-column formats: 2

Group #1: MON_YY
Count: 104
Example shapes (first 10): [(28, 4), (64, 4), (106, 4), (32, 4), (90, 4), (32, 4), (77, 4), (44, 4), (121, 4), (121, 4)]
  [Example 1] USCL | iShares Climate Conscious & Transition MSCI USA ETF | ShareClassType.OTHER
    first_col: 'Unnamed: 0'
    columns: ['Unnamed: 0', 'Fund', 'MSCI USA Index', 'MSCI USA Extended Climate Action Index']
    first_col_samples: ['May 23', 'Jun 23', 'Jul 23']
  [Example 2] USXF | iShares ESG Advanced MSCI USA ETF | ShareClassType.OTHER
    first_col: 'Unnamed: 0'
    columns: ['Unnamed: 0', 'Fund', 'MSCI USA Index', 'MSCI USA Choice ESG Screened Index']
    first_col_samples: ['May 20', 'Jun 20', 'Jul 20']
  [Example 3] ESGU | iShares ESG Aware MSCI USA ETF | ShareClassType.OTHER
    first_col: 'Unnamed: 0'
    columns: ['Unnamed: 0', 'Fund', 'MSCI USA Index', 'MSCI USA Extended ESG Focus Index (Spliced)']
    first_col_samples: ['Nov 16', 'Dec 16', 'Jan 17']
  [Example 4]

In [44]:
import sys
from pathlib import Path
%reload_ext autoreload
RAG_DIR = Path("/home/alvar/CascadeProjects/windsurf-project/RAG")
if str(RAG_DIR) not in sys.path:
    sys.path.insert(0, str(RAG_DIR))


from src.simple_rag.extraction.parser import compute_annual_returns

for fund in funds_total:
    if fund.ticker in performance_funds:
        print(fund.performance_table)
        returns = compute_annual_returns(fund.performance_table)
        print("\nFinal Annual Returns:")
        fund.annual_returns = returns
        print(f"  {fund.ticker}: {returns}")
        print("---")

   Unnamed: 0     Fund MSCI USA Index MSCI USA Extended Climate Action Index
0      May 23  $10,000        $10,000                                $10,000
1      Jun 23  $10,384        $10,994                                $10,960
2      Jul 23  $10,786        $11,374                                $11,386
3      Aug 23  $10,641        $11,180                                $11,248
4      Sep 23  $10,150        $10,656                                $10,730
5      Oct 23   $9,880        $10,410                                $10,432
6      Nov 23  $10,811        $11,391                                $11,425
7      Dec 23  $11,295        $11,927                                $11,923
8      Jan 24  $11,534        $12,122                                $12,176
9      Feb 24  $12,212        $12,085                                $12,216
10     Mar 24  $12,579        $12,467                                $13,281
11     Apr 24  $12,093        $12,630                                $12,768

In [45]:
tickers = []
unique_funds = []
duplicates = 0

for fund in funds_total:
    if fund.ticker in tickers:
        print(f"DUPLICATE: {fund.name} ({fund.ticker})")
        duplicates += 1
    else:
        tickers.append(fund.ticker)
        unique_funds.append(fund)

# Replace the original list
funds_total = unique_funds
print(f"Removed {duplicates} duplicates")
print(f"Remaining funds: {len(funds_total)}")

Removed 0 duplicates
Remaining funds: 460


In [46]:
import pickle
from pathlib import Path
import sys

# Add RAG directory to path
RAG_DIR = Path("/home/alvar/CascadeProjects/windsurf-project/RAG")
if str(RAG_DIR) not in sys.path:
    sys.path.insert(0, str(RAG_DIR))

# Define pickle file path
PKL_PATH = Path("./funds_backup_metadata.pkl")

print("Current working directory:", Path.cwd())
print("PKL_PATH resolves to:", PKL_PATH.resolve())

# Save to pickle file
try:
    with PKL_PATH.open("wb") as f:
        pickle.dump(funds_total, f)
    
    print(f"Successfully saved {len(funds_total)} funds to pickle file")
    print(f"File size: {PKL_PATH.stat().st_size / 1024:.2f} KB")
    
except Exception as e:
    print(f"Error saving to pickle file: {e}")

Current working directory: /home/luis/Desktop/code/RAG/notebooks
PKL_PATH resolves to: /home/luis/Desktop/code/RAG/notebooks/funds_backup_metadata.pkl
Successfully saved 460 funds to pickle file
File size: 4100.06 KB


In [4]:
import pickle
from pathlib import Path
from dataclasses import is_dataclass, asdict
import pandas as pd
import sys
from pathlib import Path
RAG_DIR = Path("/home/alvar/CascadeProjects/windsurf-project/RAG/")
if str(RAG_DIR) not in sys.path:
    sys.path.insert(0, str(RAG_DIR))


PKL_PATH = Path("./funds_backup.pkl")
print("Current working directory:", Path.cwd())
print("PKL_PATH resolves to:", PKL_PATH.resolve())
with PKL_PATH.open("rb") as f:
    funds_total = pickle.load(f)

print(f"Loaded {len(funds_total)} funds from pickle file")

Current working directory: /home/alvar/CascadeProjects/windsurf-project/RAG/notebooks
PKL_PATH resolves to: /home/alvar/CascadeProjects/windsurf-project/RAG/notebooks/funds_backup.pkl
Loaded 420 funds from pickle file


## Summary Prospectus

In [53]:
from edgar import Company, set_identity
import pandas as pd
from typing import List, Dict
import sys
from tqdm import tqdm
from IPython.display import display, Markdown
from src.simple_rag.extraction.general_info import FundInfoExtractor
from pathlib import Path
from src.simple_rag.models.fund import FilingMetadata
import gc
import pickle

set_identity('luis.alvarez.conde@alumnos.upm.es')
tickers = ["VOO", "MGK", "HEZU", "VMGRX", "VDIGX"]

# Create cache directory
cache_dir = Path("results/497k_cache")
cache_dir.mkdir(parents=True, exist_ok=True)

def safe_date_convert(date_value):
    """Safely convert a date value to date object, handling None/NaN."""
    if date_value is None or pd.isna(date_value):
        return None
    
    try:
        dt = pd.to_datetime(date_value)
        if pd.isna(dt):
            return None
        return dt.date()
    except Exception as e:
        print(f"Warning: Could not convert date '{date_value}': {e}")
        return None

def process_filing(filing, ticker):
    """Process a single filing and return extracted data"""
    try:
        text = filing.text()
        extractor = FundInfoExtractor(text, ticker=ticker)
        fund_data = extractor.get_structured_data()
        
        # Safely convert dates
        reporting_date = safe_date_convert(filing.report_date)
        filing_date = safe_date_convert(filing.filing_date)
       
        filing_metadata = FilingMetadata(
            accession_number=filing.accession_number,
            reporting_date=reporting_date,
            filing_date=filing_date,
            form=filing.form,
            url=filing.url
        )
        
        md = extractor.get_clean_markdown()
        
        result = {
            'ticker': fund_data['ticker'],
            'managers': fund_data['managers'],
            'strategies': fund_data['strategies'],
            'risks': fund_data['risks'],
            'objective': fund_data['objective'],
            'summary_prospectus': md,
            'filing_metadata': filing_metadata
        }
        
        # Clean up immediately
        del text, extractor, fund_data, md
        gc.collect()
        
        return result
    except Exception as e:
        print(f"❌ Error processing filing for {ticker}: {e}")
        return None

# Process each ticker
BATCH_SIZE = 5  # Process 5 filings at a time

for ticker in tickers:
    print(f"\n{'='*60}")
    print(f"Processing ticker: {ticker}")
    print(f"{'='*60}")
    
    # Check if already processed
    ticker_cache = cache_dir / f"{ticker}_processed.pkl"
    if ticker_cache.exists():
        print(f"⏭️  Skipping {ticker} - already processed (delete {ticker_cache} to reprocess)")
        continue
    
    company = Company(ticker)
    processed_funds = []
    filings = company.get_filings(form="497K")
    
    # Convert to list to enable batching
    filings_list = list(filings)
    print(f"Found {len(filings_list)} 497K filings for {ticker}")
    
    # Process filings in batches
    filing_results = []
    stop_processing = False
    
    for i in range(0, len(filings_list), BATCH_SIZE):
        if stop_processing:
            break
            
        batch = filings_list[i:i+BATCH_SIZE]
        print(f"\nProcessing batch {i//BATCH_SIZE + 1}/{(len(filings_list)-1)//BATCH_SIZE + 1}")
        
        for filing in tqdm(batch, desc=f"Batch {i//BATCH_SIZE + 1}"):
            result = process_filing(filing, ticker)
            
            if result:
                # Check for duplicates
                if result['ticker'] in processed_funds:
                    print(f"⚠️  First duplicate: {result['ticker']}")
                    stop_processing = True
                    break
                
                processed_funds.append(result['ticker'])
                filing_results.append(result)
            
            # Clear memory after each filing
            gc.collect()
        
        # Save batch progress
        batch_cache = cache_dir / f"{ticker}_batch_{i//BATCH_SIZE}.pkl"
        with open(batch_cache, 'wb') as f:
            pickle.dump(filing_results, f)
        
        print(f"✅ Saved batch progress ({len(filing_results)} filings so far)")
    
    # Mark ticker as complete
    with open(ticker_cache, 'wb') as f:
        pickle.dump({'ticker': ticker, 'count': len(processed_funds)}, f)
    
    print(f"\n✅ Processed {len(processed_funds)} funds for ticker: {ticker}")
    
    # Clear memory before next ticker
    del company, filings, filings_list, filing_results
    gc.collect()

print("\n" + "="*60)
print("All tickers processed! Now updating funds_total...")
print("="*60)

# Update funds_total with cached results
updated_count = 0

for ticker in tickers:
    ticker_cache = cache_dir / f"{ticker}_processed.pkl"
    
    if not ticker_cache.exists():
        print(f"⚠️  No cache found for {ticker}")
        continue
    
    # Load all batches for this ticker
    batch_files = sorted(cache_dir.glob(f"{ticker}_batch_*.pkl"))
    
    for batch_file in batch_files:
        with open(batch_file, 'rb') as f:
            filing_results = pickle.load(f)
        
        # Update funds_total
        for result in tqdm(filing_results, desc=f"Updating {ticker}"):
            for fund in funds_total:
                if fund.ticker == result['ticker']:
                    fund.summary_prospectus = result['summary_prospectus']
                    fund.managers = result['managers']
                    fund.strategies = result['strategies']
                    fund.risks = result['risks']
                    fund.objective = result['objective']
                    fund.summary_prospectus_metadata = result['filing_metadata']
                    updated_count += 1
                    break
        
        # Clean up
        del filing_results
        gc.collect()

print(f"\n✅ Updated {updated_count} funds in funds_total")


Processing ticker: VOO
Found 909 497K filings for VOO

Processing batch 1/182


Batch 1:   0%|          | 0/5 [00:00<?, ?it/s]

Batch 1:  20%|██        | 1/5 [00:00<00:02,  1.34it/s]

Batch 1:  40%|████      | 2/5 [00:01<00:02,  1.41it/s]

Batch 1:  60%|██████    | 3/5 [00:02<00:01,  1.44it/s]

...

Batch 3:  20%|██        | 1/5 [00:01<00:07,  1.84s/it]

⚠️  First duplicate: VGSNX
✅ Saved batch progress (11 filings so far)

✅ Processed 11 funds for ticker: VDIGX



All tickers processed! Now updating funds_total...


Updating VDIGX: 100%|██████████| 11/11 [00:00<00:00, 49292.03it/s]


✅ Updated 2061 funds in funds_total


In [51]:
from edgar import Company, set_identity
import pandas as pd
from typing import List, Dict
import sys
from tqdm import tqdm
from IPython.display import display, Markdown
from src.simple_rag.extraction.general_info import FundInfoExtractor
from pathlib import Path
from src.simple_rag.models.fund import FilingMetadata
set_identity('luis.alvarez.conde@alumnos.upm.es')
tickers = ["VOO", "MGK", "HEZU", "VMGRX", "VDIGX"]

def safe_date_convert(date_value):
    """Safely convert a date value to date object, handling None/NaN."""
    if date_value is None or pd.isna(date_value):
        return None
    
    try:
        dt = pd.to_datetime(date_value)
        if pd.isna(dt):
            return None
        return dt.date()
    except Exception as e:
        print(f"Warning: Could not convert date '{date_value}': {e}")
        return None

for ticker in tickers:
    company = Company(ticker)
    processed_funds = []
    filings = company.get_filings(form="497K")

    for filing in filings:
        text = filing.text()
        extractor = FundInfoExtractor(text, ticker=ticker)
        fund_data = extractor.get_structured_data()
        
        if fund_data['ticker'] in processed_funds:
            print("First duplicate: ", fund_data['ticker'])
            break

        # Safely convert dates
        reporting_date = safe_date_convert(filing.report_date)
        filing_date = safe_date_convert(filing.filing_date)
       
        filing_metadata = FilingMetadata(
            accession_number=filing.accession_number,
            reporting_date=reporting_date,
            filing_date=filing_date,
            form=filing.form,
            url=filing.url
        )

        processed_funds.append(fund_data['ticker'])
        md = extractor.get_clean_markdown()
        
        for fund in funds_total:
            if fund.ticker == fund_data['ticker']:
                fund.summary_prospectus = md
                fund.managers = fund_data['managers']
                fund.strategies = fund_data['strategies']
                fund.risks = fund_data['risks']
                fund.objective = fund_data['objective']
                fund.summary_prospectus_metadata = filing_metadata
                break
        
    print("Processed funds: ", len(processed_funds), "for ticker: ", ticker)

First duplicate:  VSCIX
Processed funds:  48 for ticker:  VOO


First duplicate:  VEXC
Processed funds:  38 for ticker:  MGK


ReadTimeout: The read operation timed out

In [54]:
managers = set()
for fund in funds_total:
    if fund.managers is not None:
        # If fund.managers is a list, add each manager individually
        if isinstance(fund.managers, list):
            managers.update(fund.managers)  # ✅ Use update() for lists
        else:
            managers.add(fund.managers)    # ✅ Use add() for single values

print(f"Unique managers found: {len(managers)}")
for manager in sorted(managers):
    print(f"  - {manager}")

Unique managers found: 32
  - Aaron Choi
  - Asian Economic Risk
  - Aurélie Denis
  - Brett Barakett
  - Chris Nieves
  - Christopher Chung
  - Consumer Goods
  - Erin Armstrong
  - Gary Robinson
  - Investment Manager
  - Jake Riley
  - James Mauro
  - Jena Stenger
  - Jennifer Hsui
  - Jonathan Graves
  - Kenny Narzikul
  - Lawrence Burns
  - Managing Director
  - Matt Waldron
  - Michael Cling
  - Michelle Louie
  - Nataliya Kofman
  - Natasha Kuhlkin
  - Nick Birkett
  - Peter Sietsema
  - Senior Managing Director
  - Simon Webber
  - Steven White
  - Suzanne Ly
  - Thomas Coutts
  - Tom Slater
  - Walter Nejman


In [55]:
import pickle
from pathlib import Path
import sys

# Add RAG directory to path
RAG_DIR = Path("/home/alvar/CascadeProjects/windsurf-project/RAG")
if str(RAG_DIR) not in sys.path:
    sys.path.insert(0, str(RAG_DIR))

# Define pickle file path
PKL_PATH = Path("./funds_backup_metadata.pkl")

print("Current working directory:", Path.cwd())
print("PKL_PATH resolves to:", PKL_PATH.resolve())

# Save to pickle file
try:
    with PKL_PATH.open("wb") as f:
        pickle.dump(funds_total, f)
    
    print(f"Successfully saved {len(funds_total)} funds to pickle file")
    print(f"File size: {PKL_PATH.stat().st_size / 1024:.2f} KB")
    
except Exception as e:
    print(f"Error saving to pickle file: {e}")

Current working directory: /home/luis/Desktop/code/RAG/notebooks
PKL_PATH resolves to: /home/luis/Desktop/code/RAG/notebooks/funds_backup_metadata.pkl
Successfully saved 460 funds to pickle file
File size: 11341.21 KB


In [15]:
import pickle
from pathlib import Path
from dataclasses import is_dataclass, asdict
import pandas as pd
import sys
from pathlib import Path
RAG_DIR = Path("/home/alvar/CascadeProjects/windsurf-project/RAG/")
if str(RAG_DIR) not in sys.path:
    sys.path.insert(0, str(RAG_DIR))

SRC_DIR = RAG_DIR / "src"
if str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))
    
from simple_rag.models.fund import FundData

PKL_PATH = Path("./funds_backup.pkl")
print("Current working directory:", Path.cwd())
print("PKL_PATH resolves to:", PKL_PATH.resolve())
funds_total = []
with PKL_PATH.open("rb") as f:

    funds_total = pickle.load(f)

print(f"Loaded {len(funds_total)} funds from pickle file")

Current working directory: /home/alvar/CascadeProjects/windsurf-project/RAG/notebooks
PKL_PATH resolves to: /home/alvar/CascadeProjects/windsurf-project/RAG/notebooks/funds_backup.pkl
Loaded 420 funds from pickle file


## NPORT (Portfolio Composition)

In [ ]:
from edgar import Company, set_identity
import pandas as pd
from typing import List, Dict
import sys
from tqdm import tqdm
from pathlib import Path
import gc
import pickle
from concurrent.futures import ProcessPoolExecutor, as_completed
import psutil

from src.simple_rag.extraction.nport import NPortProcessor
from src.simple_rag.models.fund import PortfolioHolding, Derivatives, NonDerivatives, FilingMetadata

company_json_path = Path("../notebooks/sec_data/company_tickers.json")

set_identity('luis.alvarez.conde@alumnos.upm.es')

# Create cache directory
cache_dir = Path("results/nport_cache")
cache_dir.mkdir(parents=True, exist_ok=True)

def safe_date_convert(date_value):
    """Safely convert a date value to date object, handling None/NaN."""
    if date_value is None or pd.isna(date_value):
        return None
    
    try:
        dt = pd.to_datetime(date_value)
        if pd.isna(dt):
            return None
        return dt.date()
    except Exception as e:
        print(f"Warning: Could not convert date '{date_value}': {e}")
        return None

def process_single_filing_worker(args):
    """Worker function for multiprocessing - processes a single filing"""
    filing_data, ticker, company_json_path = args
    
    try:
        # Re-import in worker process
        import sys
        from pathlib import Path
        sys.path.append(str(Path.cwd()))
        from src.simple_rag.extraction.nport import NPortProcessor
        from src.simple_rag.models.fund import FilingMetadata
        from edgar import Company, set_identity
        import gc
        
        set_identity('luis.alvarez.conde@alumnos.upm.es')
        
        # Reconstruct filing object from saved data
        company = Company(ticker)
        filings = company.get_filings(form="NPORT-P")
        filing = None
        
        for f in filings:
            if f.accession_number == filing_data['accession_number']:
                filing = f
                break
        
        if filing is None:
            print(f"Could not find filing {filing_data['accession_number']}")
            return None
        
        xml_data = filing.obj()
        fund_name = xml_data.get_fund_series().name
        reporting_period = xml_data.reporting_period
        portfolio_list = xml_data.investments
        derivatives = xml_data.derivatives
        series_id = xml_data.get_fund_series().series_id

        # Convert dates safely
        reporting_date = safe_date_convert(filing.report_date)
        filing_date = safe_date_convert(filing.filing_date)

        # Process holdings
        proc = NPortProcessor(company_tickers_json_path=company_json_path, min_similarity=0.74)
        holdings = proc.process_holdings(portfolio_list)
        result = proc.enrich_tickers(holdings, verbose=False)
        
        filing_metadata = FilingMetadata(
            accession_number=filing.accession_number,
            reporting_date=reporting_date,
            filing_date=filing_date,
            form=filing.form,
            url=filing.url
        )

        not_matches = result[result['matched_ticker'].isna() | (result['matched_ticker'] == '')]
        
        output = {
            'fund_name': fund_name,
            'reporting_period': reporting_period,
            'holdings': holdings,
            'result': result,
            'derivatives': derivatives,
            'not_matches': not_matches,
            'ticker': ticker,
            'series_id': series_id,
            'report_date': filing.report_date,
            'nport_metadata': filing_metadata
        }
        
        # Clean up immediately
        del xml_data, proc, holdings, result, filing
        gc.collect()
        
        return output
        
    except Exception as e:
        print(f"❌ Error processing filing {filing_data.get('accession_number', 'unknown')}: {e}")
        return None

def process_ticker_parallel(ticker, company_json_path, batch_size=3, max_workers=2):
    """Process all filings for a single ticker with parallel batch processing"""
    try:
        # Check if already processed
        ticker_cache = cache_dir / f"{ticker}_processed.pkl"
        if ticker_cache.exists():
            print(f"⏭️  Skipping {ticker} - already processed (delete to reprocess)")
            with open(ticker_cache, 'rb') as f:
                return pickle.load(f)
        
        nport_file = Company(ticker)
        filings = sorted(nport_file.get_filings(form="NPORT-P"), 
                        key=lambda x: x.report_date, reverse=True)
        
        # Convert to list for batching
        filings_list = list(filings)
        
        if not filings_list:
            print(f"No filings found for {ticker}")
            return None
            
        print(f"\n{'='*60}")
        print(f"Processing ticker: {ticker}")
        print(f"Found {len(filings_list)} filings, most recent: {filings_list[0].report_date}")
        print(f"{'='*60}")
        
        funds_processed_set = set()
        ticker_results = []
        stop_processing = False
        
        # Process filings in batches with multiprocessing
        for i in range(0, len(filings_list), batch_size):
            if stop_processing:
                break
                
            batch = filings_list[i:i+batch_size]
            print(f"\n📦 Batch {i//batch_size + 1}/{(len(filings_list)-1)//batch_size + 1}")
            
            # Prepare batch data (lightweight metadata only)
            batch_args = []
            for filing in batch:
                filing_data = {
                    'accession_number': filing.accession_number,
                    'report_date': filing.report_date
                }
                batch_args.append((filing_data, ticker, company_json_path))
            
            # Process batch in parallel
            with ProcessPoolExecutor(max_workers=max_workers) as executor:
                futures = {executor.submit(process_single_filing_worker, args): args 
                          for args in batch_args}
                
                for future in tqdm(as_completed(futures), total=len(batch_args), 
                                 desc=f"{ticker} batch {i//batch_size + 1}"):
                    result = future.result()
                    
                    if result is not None:
                        # Check if we've already processed this fund
                        if result['fund_name'].lower() in funds_processed_set:
                            print(f"  ⚠️  Stopping - already processed fund: {result['fund_name']}")
                            stop_processing = True
                            break
                        
                        funds_processed_set.add(result['fund_name'].lower())
                        ticker_results.append(result)
                        
                        print(f"  ✅ {result['fund_name']}: {len(result['holdings'])} holdings, "
                              f"{len(result['not_matches'])} unmatched")
            
            # Save batch progress
            batch_cache = cache_dir / f"{ticker}_batch_{i//batch_size}.pkl"
            with open(batch_cache, 'wb') as f:
                pickle.dump(ticker_results, f)
            
            print(f"  💾 Saved batch progress ({len(ticker_results)} filings)")
            
            # Clean memory between batches
            gc.collect()
        
        final_result = {
            'ticker': ticker,
            'results': ticker_results,
            'funds_processed': list(funds_processed_set)
        }
        
        # Save final result
        with open(ticker_cache, 'wb') as f:
            pickle.dump(final_result, f)
        
        print(f"\n✅ Completed {ticker}: {len(funds_processed_set)} funds processed")
        
        # Clean up
        del nport_file, filings, filings_list, ticker_results
        gc.collect()
        
        return final_result
    
    except Exception as e:
        print(f"❌ Error processing ticker {ticker}: {e}")
        return None

# Calculate safe worker counts
available_ram_gb = psutil.virtual_memory().available / (1024**3)
# Each NPORT worker needs ~2-3GB, be conservative
workers_per_batch = max(1, min(2, int((available_ram_gb - 4) / 3)))
batch_size = 4  # Process 4 filings per batch
workers_per_batch = 4
print(f"💻 System: {available_ram_gb:.1f}GB available RAM")
print(f"⚙️  Settings: {workers_per_batch} workers per batch, {batch_size} filings per batch")

# Main execution
tickers = ["VOO", "MGK", "HEZU", "VMGRX", "VDIGX"]

print("\n" + "="*60)
print("Processing tickers sequentially (parallel within each ticker)")
print("="*60)

all_results = []

for ticker in tickers:
    result = process_ticker_parallel(
        ticker, 
        company_json_path, 
        batch_size=batch_size,
        max_workers=workers_per_batch
    )
    if result:
        all_results.append(result)
    
    # Force cleanup between tickers
    gc.collect()

💻 System: 2.1GB available RAM
⚙️  Settings: 4 workers per batch, 4 filings per batch

Processing tickers sequentially (parallel within each ticker)

Processing ticker: VOO
Found 301 filings, most recent: 2025-09-30

📦 Batch 1/76


VOO batch 1:  25%|██▌       | 1/4 [00:01<00:05,  1.82s/it]

  ✅ VANGUARD GROWTH INDEX FUND: 166 holdings, 4 unmatched


VOO batch 1:  50%|█████     | 2/4 [00:02<00:02,  1.16s/it]

  ✅ VANGUARD MID-CAP VALUE INDEX FUND: 186 holdings, 4 unmatched


...

VDIGX batch 2:  25%|██▌       | 1/4 [00:01<00:05,  1.85s/it]

  ✅ VANGUARD GLOBAL ESG SELECT STOCK FUND: 38 holdings, 7 unmatched
  ⚠️  Stopping - already processed fund: VANGUARD ENERGY FUND


  💾 Saved batch progress (5 filings)

✅ Completed VDIGX: 5 funds processed

All tickers processed! Now updating funds_total...


Updating VDIGX: 100%|██████████| 5/5 [00:00<00:00,  8.64it/s]


✅ Updated 0 funds in funds_total


In [10]:
import pickle
import os
from pathlib import Path

# Specify the folder path
folder_path = './results/nport_cache'  # Update this path

# Initialize an empty list to store all objects
all_objects = []

# Get all files ending with '_processed.pickle' (or '_processed.pkl')
folder = Path(folder_path)
processed_files = list(folder.glob('*_processed.pickle')) + list(folder.glob('*_processed.pkl'))

# Read each pickle file and add objects to the array
for file_path in processed_files:
    try:
        with open(file_path, 'rb') as file:
            data = pickle.load(file)
            # If data is already a list, extend; otherwise append
            if isinstance(data, list):
                all_objects.extend(data)
            else:
                all_objects.append(data)
        print(f"Loaded: {file_path.name}")
    except Exception as e:
        print(f"Error loading {file_path.name}: {e}")

print(f"\nTotal objects loaded: {len(all_objects)}")

Loaded: VOO_processed.pkl
Loaded: MGK_processed.pkl
Loaded: VMGRX_processed.pkl
Loaded: VDIGX_processed.pkl
Loaded: HEZU_processed.pkl

Total objects loaded: 5


In [24]:
for result in all_objects:
    print(result['results'][0]['result'])
    break

                                          holding_name ticker_before  \
0                                          ABIOMED Inc          None   
1                                            Adobe Inc          None   
2                           Advanced Micro Devices Inc          None   
3                                           Airbnb Inc          None   
4                          Alnylam Pharmaceuticals Inc          None   
..                                                 ...           ...   
161                                                N/A          NQZ5   
162  Vanguard Cmt Funds-Vanguard Market Liquidity Fund          None   
163                                                N/A          None   
164                                                N/A          None   
165                        Royal Caribbean Cruises Ltd          None   

    ticker_after matched_ticker                  matched_title  similarity  \
0           None           None                          

In [ ]:
from edgar import Company, set_identity
import pandas as pd
from typing import List, Dict
import sys
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import multiprocessing as mp
from threading import Lock
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
from src.simple_rag.extraction.nport import NPortProcessor
from src.simple_rag.models.fund import PortfolioHolding, Derivatives, NonDerivatives, FilingMetadata
from pathlib import Path

company_json_path = Path("/home/alvar/CascadeProjects/windsurf-project/RAG/notebooks/sec_data/company_tickers.json")

set_identity('luis.alvarez.conde@alumnos.upm.es')

def process_single_filing(filing, ticker, company_json_path):
    """Process a single filing - can be parallelized"""
    try:
        xml_data = filing.obj()
        fund_name = xml_data.get_fund_series().name
        reporting_period = xml_data.reporting_period
        portfolio_list = xml_data.investments
        derivatives = xml_data.derivatives
        series_id = xml_data.get_fund_series().series_id

        # Process holdings
        proc = NPortProcessor(company_tickers_json_path=company_json_path, min_similarity=0.74)
        holdings = proc.process_holdings(portfolio_list)
        result = proc.enrich_tickers(holdings, verbose=False)  # Set verbose=False to reduce I/O
        filing_metadata = FilingMetadata(
            accession_number=filing.accession_number,
            reporting_date=reporting_date,
            filing_date=filing_date,
            form=filing.form,
            url=filing.url
        )

        not_matches = result[result['matched_ticker'].isna() | (result['matched_ticker'] == '')]
        
        return {
            'fund_name': fund_name,
            'reporting_period': reporting_period,
            'holdings': holdings,
            'result': result,
            'derivatives': derivatives,
            'not_matches': not_matches,
            'ticker': ticker,
            'series_id': series_id,
            'report_date': filing.report_date,
            'nport_metadata': filing_metadata
        }
    except Exception as e:
        print(f"Error processing filing for {ticker}: {e}")
        return None

def process_ticker(ticker, company_json_path):
    """Process all filings for a single ticker - SEQUENTIAL within ticker"""
    try:
        nport_file = Company(ticker)
        filings = sorted(nport_file.get_filings(form="NPORT-P"), 
                        key=lambda x: x.report_date, reverse=True)
        
        if not filings:
            print(f"No filings found for {ticker}")
            return None
            
        print(f"Processing ticker: {ticker}, most recent filing date: {filings[0].report_date}")
        
        funds_processed_set = set()
        ticker_results = []
        
        # Process filings SEQUENTIALLY for this ticker (to respect the stop condition)
        for filing in filings:
            result = process_single_filing(filing, ticker, company_json_path)
            
            if result is not None:
                # Check if we've already processed this fund
                if result['fund_name'].lower() in funds_processed_set:
                    print(f"Stopping - already processed fund: {result['fund_name']}")
                    break
                
                funds_processed_set.add(result['fund_name'].lower())
                ticker_results.append(result)
                
                print(f"{ticker} - Fund: {result['fund_name']}, Holdings: {len(result['holdings'])}, Unmatched: {len(result['not_matches'])}")
        
        return {
            'ticker': ticker,
            'results': ticker_results,
            'funds_processed': list(funds_processed_set)
        }
    
    except Exception as e:
        print(f"Error processing ticker {ticker}: {e}")
        return None

# Main execution - PARALLEL across tickers only
tickers = ["VOO", "MGK", "HEZU", "VMGRX", "VDIGX"]

# Use fewer workers to avoid overwhelming the system
max_workers = min(5, len(tickers))  # Start with 3 workers
print(f"Using {max_workers} workers for tickers")

all_results = []
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    future_to_ticker = {
        executor.submit(process_ticker, ticker, company_json_path): ticker 
        for ticker in tickers
    }
    
    for future in tqdm(as_completed(future_to_ticker), total=len(tickers), desc="Processing tickers"):
        ticker = future_to_ticker[future]
        try:
            result = future.result()
            if result:
                all_results.append(result)
                print(f"\nCompleted {ticker}: {len(result['funds_processed'])} funds processed")
        except Exception as e:
            print(f"Error with ticker {ticker}: {e}")

# Update funds_total object with the results
print("\n=== Updating funds_total ===")


/home/luis/Desktop/code/RAG/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using 5 workers for tickers


Processing tickers:   0%|          | 0/5 [00:00<?, ?it/s]

Processing ticker: VDIGX, most recent filing date: 2025-10-31
Processing ticker: MGK, most recent filing date: 2025-11-30
Error processing filing for VDIGX: [Errno 2] No such file or directory: '/home/alvar/CascadeProjects/windsurf-project/RAG/notebooks/sec_data/company_tickers.json'
Error processing filing for VDIGX: [Errno 2] No such file or directory: '/home/alvar/CascadeProjects/windsurf-project/RAG/notebooks/sec_data/company_tickers.json'
Processing ticker: VOO, most recent filing date: 2025-09-30
Error processing filing for VDIGX: [Errno 2] No such file or directory: '/home/alvar/CascadeProjects/windsurf-project/RAG/notebooks/sec_data/company_tickers.json'
Error processing filing for VOO: [Errno 2] No such file or directory: '/home/alvar/CascadeProjects/windsurf-project/RAG/notebooks/sec_data/company_tickers.json'
Error processing filing for VDIGX: [Errno 2] No such file or directory: '/home/alvar/CascadeProjects/windsurf-project/RAG/notebooks/sec_data/company_tickers.json'
Error

Processing tickers:   0%|          | 0/5 [00:09<?, ?it/s]

Error processing filing for VMGRX: [Errno 2] No such file or directory: '/home/alvar/CascadeProjects/windsurf-project/RAG/notebooks/sec_data/company_tickers.json'
Error processing filing for VDIGX: [Errno 2] No such file or directory: '/home/alvar/CascadeProjects/windsurf-project/RAG/notebooks/sec_data/company_tickers.json'


Error processing filing for VOO: [Errno 2] No such file or directory: '/home/alvar/CascadeProjects/windsurf-project/RAG/notebooks/sec_data/company_tickers.json'
Error processing filing for VMGRX: [Errno 2] No such file or directory: '/home/alvar/CascadeProjects/windsurf-project/RAG/notebooks/sec_data/company_tickers.json'
Error processing filing for VOO: [Errno 2] No such file or directory: '/home/alvar/CascadeProjects/windsurf-project/RAG/notebooks/sec_data/company_tickers.json'
Error processing filing for VDIGX: [Errno 2] No such file or directory: '/home/alvar/CascadeProjects/windsurf-project/RAG/notebooks/sec_data/company_tickers.json'
Error processing filing for VMGRX: [Errno 2] No such file or directory: '/home/alvar/CascadeProjects/windsurf-project/RAG/notebooks/sec_data/company_tickers.json'
Error processing filing for VOO: [Errno 2] No such file or directory: '/home/alvar/CascadeProjects/windsurf-project/RAG/notebooks/sec_data/company_tickers.json'
Error processing filing for 

In [4]:
for fund in funds_total:
    print(fund.name)
    if "vanguard" not in fund.name.lower() and "ishares" not in fund.name.lower():
        fund.name = "Vanguard " + fund.name
        

Vanguard Extended Market Index Fund
Vanguard Extended Market Index Fund
Vanguard Extended Market Index Fund
Vanguard Extended Market Index Fund
Vanguard Extended Market Index Fund
Vanguard Extended Market Index Fund
Vanguard Mid-Cap Index Fund
Vanguard Mid-Cap Index Fund
Vanguard Mid-Cap Index Fund
Vanguard Mid-Cap Index Fund
Vanguard Mid-Cap Index Fund
Vanguard Mid-Cap Growth Index Fund
Vanguard Mid-Cap Growth Index Fund
Vanguard Mid-Cap Growth Index Fund
Vanguard Mid-Cap Value Index Fund
Vanguard Mid-Cap Value Index Fund
Vanguard Mid-Cap Value Index Fund
Vanguard Small-Cap Index Fund
Vanguard Small-Cap Index Fund
Vanguard Small-Cap Index Fund
Vanguard Small-Cap Index Fund
Vanguard Small-Cap Index Fund
Vanguard Small-Cap Growth Index Fund
Vanguard Small-Cap Growth Index Fund
Vanguard Small-Cap Growth Index Fund
Vanguard Small-Cap Growth Index Fund
Vanguard Small-Cap Value Index Fund
Vanguard Small-Cap Value Index Fund
Vanguard Small-Cap Value Index Fund
Vanguard Small-Cap Value Index 

In [25]:
print("="*60)
print("SIMPLE NAME MATCHING ANALYSIS")
print("="*60)

unmatched_count = 0
matched_count = 0
unmateched_funds = []

for ticker_result in all_objects:
    for filing in ticker_result['results']:
        filing_name = filing['fund_name']
        print(filing_name)
        
        # We will collect all "candidates" from your funds_total list
        found_candidates = []
        
        for fund in funds_total:
            # 1. Clean both names (lowercase, strip whitespace)
            f_name = filing_name.lower().strip()
            db_name = fund.name.lower().strip()
            
            # 2. Check: Is one inside the other?
            # We check both directions: "Vanguard 500" in "Vanguard 500 Index" OR vice versa
            if f_name in db_name:
                found_candidates.append(fund)
            
        # --- PRINT RESULTS ---
        if not found_candidates:
            unmatched_count += 1
            unmateched_funds.append(filing)
            print(f"❌ NO MATCH FOUND")
            print(f"   Filing Name: '{filing_name}'")
            print("   (Check: Is the spelling significantly different?)")
            print("-" * 40)
        else:
            matched_count += 1
            # Only print if you want to verify the logic (optional: comment out to reduce noise)
            print(f"✅  MATCHED: '{filing_name}'")
            print(f"   -> Linked to {len(found_candidates)} funds in DB:")
            found_name = found_candidates[0].name
            for c in found_candidates:
                if c.name == found_name:
                    print(f"      • {c.ticker} | {c.name}")
                    c.series_id = filing['series_id']
            print("-" * 40)

print("\n" + "="*30)
print("SUMMARY")
print(f"Total Filings Processed: {matched_count + unmatched_count}")
print(f"Matches Found:           {matched_count}")
print(f"Unmatched:               {unmatched_count}")
print("="*30)

SIMPLE NAME MATCHING ANALYSIS
VANGUARD GROWTH INDEX FUND
✅  MATCHED: 'VANGUARD GROWTH INDEX FUND'
   -> Linked to 4 funds in DB:
      • VIGRX | Vanguard Growth Index Fund
      • VUG | Vanguard Growth Index Fund
      • VIGAX | Vanguard Growth Index Fund
      • VIGIX | Vanguard Growth Index Fund
----------------------------------------
VANGUARD MID-CAP VALUE INDEX FUND
✅  MATCHED: 'VANGUARD MID-CAP VALUE INDEX FUND'
   -> Linked to 3 funds in DB:
      • VMVIX | Vanguard Mid-Cap Value Index Fund
      • VOE | Vanguard Mid-Cap Value Index Fund
      • VMVAX | Vanguard Mid-Cap Value Index Fund
----------------------------------------
VANGUARD SMALL-CAP INDEX FUND
✅  MATCHED: 'VANGUARD SMALL-CAP INDEX FUND'
   -> Linked to 5 funds in DB:
      • NAESX | Vanguard Small-Cap Index Fund
      • VB | Vanguard Small-Cap Index Fund
      • VSMAX | Vanguard Small-Cap Index Fund
      • VSCIX | Vanguard Small-Cap Index Fund
      • VSCPX | Vanguard Small-Cap Index Fund
--------------------------

In [9]:
for fund in funds_total:
    if fund.non_derivatives is not None:
        print(fund.series_id)
        break

In [26]:

processor = NPortProcessor()
for ticker_result in all_objects:
    
    for filing_result in ticker_result['results']:
        print(f"Processing ticker: {filing_result['ticker']} {filing_result['fund_name']}")
        print(filing_result.keys())
        ticker = filing_result['ticker']
        reporting_period = filing_result['reporting_period']
        holdings = filing_result['result']
        derivatives = filing_result['derivatives']
        series_id = filing_result['series_id']
        filing_metadata = filing_result['nport_metadata']
        # Update your funds_total structure
        for fund in funds_total:
            if hasattr(fund, 'series_id'):
                if series_id == fund.series_id:
                    print(f"Updating fund: {fund.name}")
                    df = processor.to_df(holdings)

                    fund.non_derivatives = NonDerivatives(
                    date=reporting_period,
                    holdings_df=df
                    )
                    df2 = processor.to_df(derivatives)
                    fund.derivatives = Derivatives(
                        date=reporting_period,
                        derivatives_df=df2
                    )
                    fund.series_id = series_id
                    fund.nport_metadata = filing_metadata
                    break

print("\n=== Processing Complete ===")
print(f"Total tickers processed: {len(all_results)}")
for result in all_results:
    print(f"{result['ticker']}: {len(result['funds_processed'])} funds")



def verify_fund_data_integrity(funds_list):
    """
    Iterates through funds to verify that the holdings DataFrame is populated 
    and counts how many are None or missing.
    """
    print("\n" + "="*40)
    print("DATA INTEGRITY VERIFICATION")
    print("="*40)
    
    none_count = 0
    valid_count = 0
    
    for fund in funds_list:
        # We need to check layers: 
        # 1. Does fund.non_derivatives exist? 
        # 2. Is it not None?
        # 3. Is the holdings_df inside it not None?
        
        has_data = False
        row_count = 0
        
        try:
            if (hasattr(fund, 'non_derivatives') and 
                fund.non_derivatives is not None and 
                fund.non_derivatives.holdings_df is not None):
                
                has_data = True
                row_count = len(fund.non_derivatives.holdings_df)
        except Exception:
            # If any attribute access fails, treat as no data
            has_data = False

        if has_data:
            valid_count += 1
            # Optional: Print success if you want to see the good ones
            # print(f"✓ {fund.name:<20} | Rows: {row_count}")
        else:
            none_count += 1
            print(f"❌ {fund.name:<20} | Status: DATAFRAME IS NONE/MISSING")

    print("-" * 40)
    print(f"Total Funds Checked: {len(funds_list)}")
    print(f"Valid DataFrames:    {valid_count}")
    print(f"None/Missing Values: {none_count}")
    print("=" * 40)

# Run the verification
# Note: Ensure 'funds_total' is defined in your scope (it was referenced in your loop)
if 'funds_total' in locals():
    verify_fund_data_integrity(funds_total)
else:
    print("Error: 'funds_total' variable not found in current scope.")

Processing ticker: VOO VANGUARD GROWTH INDEX FUND
dict_keys(['fund_name', 'reporting_period', 'holdings', 'result', 'derivatives', 'not_matches', 'ticker', 'series_id', 'report_date', 'nport_metadata'])
Updating fund: Vanguard Growth Index Fund
Processing ticker: VOO VANGUARD MID-CAP VALUE INDEX FUND
dict_keys(['fund_name', 'reporting_period', 'holdings', 'result', 'derivatives', 'not_matches', 'ticker', 'series_id', 'report_date', 'nport_metadata'])
Updating fund: Vanguard Mid-Cap Value Index Fund
Processing ticker: VOO VANGUARD SMALL-CAP INDEX FUND
dict_keys(['fund_name', 'reporting_period', 'holdings', 'result', 'derivatives', 'not_matches', 'ticker', 'series_id', 'report_date', 'nport_metadata'])
Updating fund: Vanguard Small-Cap Index Fund
Processing ticker: VOO VANGUARD EXTENDED MARKET INDEX FUND
dict_keys(['fund_name', 'reporting_period', 'holdings', 'result', 'derivatives', 'not_matches', 'ticker', 'series_id', 'report_date', 'nport_metadata'])
Updating fund: Vanguard Extended 

In [30]:
for fund in funds_total:
    if fund.non_derivatives is None:
        continue
    print(fund.objective)
    break


The Fund seeks to track the performance of a benchmark index that measuresthe investment return of small- and mid-capitalization stocks.


In [29]:
import pickle
from pathlib import Path

PKL_PATH = Path("./funds_backup_metadata.pkl")
TMP_PATH = PKL_PATH.with_suffix(PKL_PATH.suffix + ".tmp")

with TMP_PATH.open("wb") as f:
    pickle.dump(funds_total, f, protocol=pickle.HIGHEST_PROTOCOL)

TMP_PATH.replace(PKL_PATH)

print(f"Saved {len(funds_total)} funds to pickle file: {PKL_PATH.resolve()}")
print(f"File size: {PKL_PATH.stat().st_size / (1024 * 1024):.2f} MB")

Saved 460 funds to pickle file: /home/luis/Desktop/code/RAG/notebooks/funds_backup_metadata.pkl
File size: 11.58 MB


In [1]:
import pickle
from pathlib import Path
from dataclasses import is_dataclass, asdict
import pandas as pd
import sys
from pathlib import Path
RAG_DIR = Path("/home/luis/Desktop/code/RAG/src/")
if str(RAG_DIR) not in sys.path:
    sys.path.insert(0, str(RAG_DIR))


PKL_PATH = Path("./funds_backup_metadata.pkl")
print("Current working directory:", Path.cwd())
print("PKL_PATH resolves to:", PKL_PATH.resolve())
with PKL_PATH.open("rb") as f:
    funds_total = pickle.load(f)

print(f"Loaded {len(funds_total)} funds from pickle file")

Current working directory: /home/luis/Desktop/code/RAG/notebooks
PKL_PATH resolves to: /home/luis/Desktop/code/RAG/notebooks/funds_backup_metadata.pkl
Loaded 460 funds from pickle file


## Company Filings

## Processing Phase

### General information about the fund

In [31]:

for fund in funds_total:
    if 'Vanguard' in fund.name:
        fund.provider = 'The Vanguard Group, Inc'
    else:
        fund.provider = 'BlackRock, Inc'

In [32]:
registrants = set()
for fund in funds_total:
    if '\xa0' in fund.registrant:
        fund.registrant = fund.registrant.replace('\xa0', ' ')
    registrants.add(fund.registrant)

print(registrants)

{'Vanguard Whitehall Funds', 'Vanguard World Fund', 'Vanguard Specialized Funds', 'Vanguard Index Funds', 'iShares Trust'}


In [33]:
from src.simple_rag.models.fund import ShareClassType

unique_share_classes = set(fund.share_class for fund in funds_total)
print("Unique Share Classes:")
for share_class in unique_share_classes:
    print(share_class)

from collections import Counter

# Count funds by share class
share_counts = Counter()
for fund in funds_total:
    if fund.share_class:
        if fund.share_class == ShareClassType.OTHER:
            fund.share_class = ShareClassType.ETF
        share_counts[fund.share_class] += 1

print("Funds by share class:")
for share_type, count in share_counts.items():
    print(f"  {share_type.value}: {count} funds")



Unique Share Classes:
ShareClassType.INSTITUTIONAL
ShareClassType.ADMIRAL
ShareClassType.OTHER
ShareClassType.ETF
ShareClassType.INSTITUTIONAL_SELECT
ShareClassType.INVESTOR
ShareClassType.INSTITUTIONAL_PLUS
Funds by share class:
  Investor Shares: 27 funds
  ETF Shares: 370 funds
  Admiral Shares: 40 funds
  Institutional Shares: 16 funds
  Institutional Plus Shares: 4 funds
  Institutional Select Shares: 3 funds


In [34]:
from collections import Counter
import unicodedata

def normalize_name(text):
    if not text:
        return "Unknown"
    # NFKD normalization turns non-breaking spaces into normal spaces
    return unicodedata.normalize("NFKD", text).strip()

# Your original loop, but with the fix applied:
share_counts = Counter()

for fund in funds_total:
    if fund.registrant:
        # Apply the fix here
        clean_registrant = normalize_name(fund.registrant)
        share_counts[clean_registrant] += 1

print("Funds by share class (Merged):")
for share_type, count in share_counts.items():
    print(f"  {share_type}: {count} funds")

Funds by share class (Merged):
  Vanguard Index Funds: 52 funds
  Vanguard World Fund: 41 funds
  Vanguard Specialized Funds: 14 funds
  Vanguard Whitehall Funds: 18 funds
  iShares Trust: 335 funds


In [35]:
from collections import Counter
import unicodedata

def normalize_name(text):
    if not text:
        return "Unknown"
    # NFKD normalization turns non-breaking spaces into normal spaces
    return unicodedata.normalize("NFKD", text).strip()

fund_counts = Counter()

for fund in funds_total:
    if fund.name:
        # 1. Normalize the name first
        clean_name = normalize_name(fund.name)
        
        # 2. Check if it starts with iShares or Vanguard
        # We check case-sensitively, but you could use .lower() if needed
        if not (clean_name.startswith("iShares") or clean_name.startswith("Vanguard")):
            # 3. Add 'Vanguard' if missing
            clean_name = f"Vanguard {clean_name}"
            
        # 4. Count the final modified name
        fund_counts[clean_name] += 1

print("Funds by name (Merged & Fixed):")
for fund_name, count in fund_counts.items():
    print(f"  {fund_name}: {count} funds")



Funds by name (Merged & Fixed):
  Vanguard Extended Market Index Fund: 6 funds
  Vanguard Mid-Cap Index Fund: 5 funds
  Vanguard Mid-Cap Growth Index Fund: 3 funds
  Vanguard Mid-Cap Value Index Fund: 3 funds
  Vanguard Small-Cap Index Fund: 5 funds
  Vanguard Small-Cap Growth Index Fund: 4 funds
  Vanguard Small-Cap Value Index Fund: 4 funds
  Vanguard Total Stock Market Index Fund: 6 funds
  Vanguard 500 Index Fund: 4 funds
  Vanguard Value Index Fund: 4 funds
  Vanguard Growth Index Fund: 4 funds
  Vanguard Large-Cap Index Fund: 4 funds
  Vanguard Mega Cap Growth Index Fund: 2 funds
  Vanguard Extended Duration Treasury Index Fund: 2 funds
  Vanguard ESG U.S. Stock ETF: 1 funds
  Vanguard ESG International Stock ETF: 1 funds
  Vanguard Global WellingtonTM Fund: 2 funds
  Vanguard Global Wellesley® Income Fund: 2 funds
  Vanguard ESG U.S. Corporate Bond ETF: 1 funds
  Vanguard FTSE Social Index Fund: 2 funds
  Vanguard Materials Index Fund: 2 funds
  Vanguard Communication Services I

In [36]:
sum = 0
for fund in funds_total:
    # Format correctly the numeric fields
    fund.net_assets *= 1e6
    fund.advisory_fees *= 1e3
   

In [37]:
from src.simple_rag.utils.chart_utils import extract_flexible_performance
from src.simple_rag.models.fund import AverageReturnSnapshot
for fund in funds_total:
    
    year = fund.report_date.year
    if fund.avg_annual_returns is None:
        continue 
    result = extract_flexible_performance(fund.avg_annual_returns)
    
    # Create AverageReturnSnapshot instance
    snapshot = AverageReturnSnapshot(
        return_1y=result.get('1_year', None),
        return_5y=result.get('5_year', None),
        return_10y=result.get('10_year', None),
        return_inception=result.get('since_inception', None)
    )
    
    fund.performance[str(year)] = snapshot 
    

In [38]:
for fund in funds_total:
    if fund.financial_highlights is not None:
        print(fund.financial_highlights)
        for _, values in fund.financial_highlights.items():
            values.net_assets = values.net_assets * 1e6

{'2024': FinancialHighlights(turnover=11.0, expense_ratio=0.19, total_return=16.76, net_assets=195.0, net_assets_value_begining=124.78, net_assets_value_end=144.2, net_income_ratio=1.09), '2023': FinancialHighlights(turnover=11.0, expense_ratio=0.19, total_return=25.22, net_assets=232.0, net_assets_value_begining=100.93, net_assets_value_end=124.78, net_income_ratio=1.28), '2022': FinancialHighlights(turnover=11.0, expense_ratio=0.19, total_return=-26.56, net_assets=229.0, net_assets_value_begining=138.8, net_assets_value_end=100.93, net_income_ratio=1.14), '2021': FinancialHighlights(turnover=19.0, expense_ratio=0.19, total_return=12.31, net_assets=399.0, net_assets_value_begining=124.83, net_assets_value_end=138.8, net_income_ratio=0.87), '2020': FinancialHighlights(turnover=19.0, expense_ratio=0.19, total_return=32.04, net_assets=454.0, net_assets_value_begining=95.66, net_assets_value_end=124.83, net_income_ratio=1.04)}
{'2024': FinancialHighlights(turnover=11.0, expense_ratio=0.06

In [39]:
managers_set = set()
for fund in funds_total:
    if hasattr(fund, 'managers') and fund.managers is not None:
        managers_set.update(fund.managers)
for manager in sorted(managers_set):
    print(manager)

Aaron Choi
Asian Economic Risk
Aurélie Denis
Brett Barakett
Chris Nieves
Christopher Chung
Consumer Goods
Erin Armstrong
Gary Robinson
Investment Manager
Jake Riley
James Mauro
Jena Stenger
Jennifer Hsui
Jonathan Graves
Kenny Narzikul
Lawrence Burns
Managing Director
Matt Waldron
Michael Cling
Michelle Louie
Nataliya Kofman
Natasha Kuhlkin
Nick Birkett
Peter Sietsema
Senior Managing Director
Simon Webber
Steven White
Suzanne Ly
Thomas Coutts
Tom Slater
Walter Nejman


Enhance the summary prospectus 

In [40]:
def update_summary_prospectus_with_header(fund):
    """Remove all existing headers and add new header."""
    
    header = f"# FUND PROFILE({fund.ticker} - {fund.name} - {fund.share_class if fund.share_class else 'N/A'}):"
    
    if not fund.summary_prospectus or not fund.summary_prospectus.strip():
        return header
    
    # Remove all header lines and keep content
    lines = fund.summary_prospectus.split('\n')
    content_lines = [line for line in lines if not line.strip().startswith("# FUND PROFILE(")]
    
    # Add new header at beginning with proper spacing
    if content_lines:
        return f"{header}\n" + '\n'.join(content_lines)
    else:
        return header

# Update all funds
for i, fund in enumerate(funds_total):
    fund.summary_prospectus = update_summary_prospectus_with_header(fund)
    print(f"✅ {i+1}/{len(funds_total)}: Updated {fund.ticker}")

✅ 1/460: Updated VEXMX
✅ 2/460: Updated VXF
✅ 3/460: Updated VEXAX
✅ 4/460: Updated VIEIX
✅ 5/460: Updated VEMPX
✅ 6/460: Updated VSEMX
✅ 7/460: Updated VIMSX
✅ 8/460: Updated VO
✅ 9/460: Updated VIMAX
✅ 10/460: Updated VMCIX
✅ 11/460: Updated VMCPX
✅ 12/460: Updated VMGIX
✅ 13/460: Updated VOT
✅ 14/460: Updated VMGMX
✅ 15/460: Updated VMVIX
✅ 16/460: Updated VOE
✅ 17/460: Updated VMVAX
✅ 18/460: Updated NAESX
✅ 19/460: Updated VB
✅ 20/460: Updated VSMAX
✅ 21/460: Updated VSCIX
✅ 22/460: Updated VSCPX
✅ 23/460: Updated VISGX
✅ 24/460: Updated VBK
✅ 25/460: Updated VSGAX
✅ 26/460: Updated VSGIX
✅ 27/460: Updated VISVX
✅ 28/460: Updated VBR
✅ 29/460: Updated VSIAX
✅ 30/460: Updated VSIIX
✅ 31/460: Updated VTSMX
✅ 32/460: Updated VTI
✅ 33/460: Updated VTSAX
✅ 34/460: Updated VITSX
✅ 35/460: Updated VSMPX
✅ 36/460: Updated VSTSX
✅ 37/460: Updated VFINX
✅ 38/460: Updated VOO
✅ 39/460: Updated VFIAX
✅ 40/460: Updated VFFSX
✅ 41/460: Updated VIVAX
✅ 42/460: Updated VTV
✅ 43/460: Updated VVIAX

In [41]:
# Check a few funds to see the result
for fund in funds_total[:3]:
    print(f"\n{fund.ticker}:")
    print(f"First 100 chars: ...")
    print(fund.summary_prospectus)
    print(f"Has header: {'# FUND PROFILE(' in fund.summary_prospectus}")


VEXMX:
First 100 chars: ...
# FUND PROFILE(VEXMX - Vanguard Extended Market Index Fund - ShareClassType.INVESTOR):

## Investment Objective
The Fund seeks to track the performance of a benchmark index that measuresthe investment return of small- and mid-capitalization stocks.

## Principal Investment Strategies
The Fund employs an indexing investment approach designed to track theperformance of the S& P Completion Index (the Index), a broadly diversified indexof stocks of small and mid-size U. S. companies. The Index contains all of theU. S. common stocks regularly traded on the New York Stock Exchange, Cboe, and the Nasdaq over-the-counter market, except those stocks included in theS& P 500 Index. The Fund invests by sampling the Index, meaning that it holds abroadly diversified collection of securities that, in the aggregate, approximatesthe full Index in terms of key characteristics. These characteristics includeindustry weightings and market capitalization, as well as certain fina

### Geographic Allocation

In [42]:

for fund in funds_total:
    df = fund.geographic_allocation
    if df is not None:
        df = df.iloc[:,[0,1]]
        df.columns = ['Country', 'Percentage']
        df.iloc[:, 0] = df.iloc[:, 0].astype(str).str.replace('\u200b', '', regex=True).str.strip()
        df.iloc[:, 0] = df.iloc[:, 0].replace(['', 'nan', 'None', 'Other#', 'Country/Geographic Region'], 'Other')
        df.iloc[:, 1] = pd.to_numeric(df.iloc[:, 1], errors='coerce')
        df = df[df.iloc[:, 1].notna()]
        
        fund.geographic_allocation = df
        
countries = set()
for fund in funds_total:
    df = fund.geographic_allocation
    if df is not None:
        # Get unique values from the first column
        unique_values = df.iloc[:, 0].unique()
        countries.update(unique_values)
print(f"Unique countries: {len(countries)}")
print("Countries:", sorted(countries))
# Verify data types
for fund in funds_total:  # Check first fund
    if fund.geographic_allocation is not None:
        print(f"\nSample cleaned data for {fund.ticker}:")
        print(f"Shape: {fund.geographic_allocation.shape}")
        print(f"Data types: {fund.geographic_allocation.dtypes}")
        print(f"Sample rows:")
        print(fund.geographic_allocation)
        break


Unique countries: 42
Countries: ['Australia', 'Belgium', 'Brazil', 'Canada', 'Chile', 'China', 'Colombia', 'Denmark', 'Finland', 'France', 'Germany', 'Hong Kong', 'India', 'Indonesia', 'Israel', 'Italy', 'Japan', 'Malaysia', 'Mexico', 'Netherlands', 'New Zealand', 'Norway', 'Other', 'Peru', 'Philippines', 'Poland', 'Portugal', 'Qatar', 'Saudi Arabia', 'Singapore', 'South Africa', 'South Korea', 'Spain', 'Supranational', 'Sweden', 'Switzerland', 'Taiwan', 'Thailand', 'Turkey', 'United Arab Emirates', 'United Kingdom', 'United States']

Sample cleaned data for DMXF:
Shape: (11, 2)
Data types: Country       object
Percentage    object
dtype: object
Sample rows:
           Country Percentage
1            Japan       25.1
2      Switzerland       11.8
3           France       10.3
4      Netherlands        8.6
5          Germany        8.2
6   United Kingdom        7.5
7           Sweden        6.1
8        Australia        4.5
9        Hong Kong        3.3
10       Singapore        2.6
11 

### Sector Allocation


In [43]:
from src.simple_rag.utils.chart_utils import validate_and_clean_allocation

for fund in funds_total:
    df = fund.sector_allocation
    if df is not None:
        df = validate_and_clean_allocation(df, 'Sector', sort_by_value=True)
        fund.sector_allocation = df


VALIDATION REPORT
Category Column: 'Sector' | Value Column: 'Percent of Total Investments(a)'
❌ Row  0: 'Portfolio Composition % of Net' | 'nan     ' | Numeric: True | Range OK: False
✅ Row  1: 'Communication Services        ' | '4.3%    ' | Numeric: True | Range OK: True
✅ Row  2: 'Consumer Discretionary        ' | '12.0%   ' | Numeric: True | Range OK: True
✅ Row  3: 'Consumer Staples              ' | '3.0%    ' | Numeric: True | Range OK: True
✅ Row  4: 'Energy                        ' | '4.1%    ' | Numeric: True | Range OK: True
✅ Row  5: 'Financials                    ' | '18.0%   ' | Numeric: True | Range OK: True
✅ Row  6: 'Health Care                   ' | '11.4%   ' | Numeric: True | Range OK: True
✅ Row  7: 'Industrials                   ' | '17.4%   ' | Numeric: True | Range OK: True
✅ Row  8: 'Information Technology        ' | '17.9%   ' | Numeric: True | Range OK: True
✅ Row  9: 'Materials                     ' | '4.7%    ' | Numeric: True | Range OK: True
✅ Row 10: 'Real

In [44]:
import pandas as pd
import re

# 1. Get all sectors from your funds
sectors = set()
for fund in funds_total:
    df = fund.sector_allocation
    if df is not None:
        sectors.update(df.iloc[:, 0].unique())

# 2. The Cleaning Function
def get_standard_key(text):
    if not isinstance(text, str):
        return text
        
    # A. Convert to lowercase
    clean = text.lower()
    
    # B. Remove the specific word "sector"
    clean = re.sub(r'\bsector\b', '', clean)
    
    # C. Remove punctuation
    clean = re.sub(r'[^a-z0-9\s]', '', clean)
    
    # D. Remove extra whitespace
    clean = re.sub(r'\s+', ' ', clean).strip()
    
    return clean

# 3. Create mapping DataFrame
sectors_list = list(sectors)
df_mapping = pd.DataFrame({'Original': sectors_list})
df_mapping['Clean_Key'] = df_mapping['Original'].apply(get_standard_key)

# 4. Create canonical mapping (pick shortest name as standard)
canonical_map = df_mapping.groupby('Clean_Key')['Original'].transform(lambda x: sorted(x, key=len)[0])
df_mapping['Standardized'] = canonical_map

# 5. Create the mapping dictionary
standardization_dict = dict(zip(df_mapping['Original'], df_mapping['Standardized']))

# 6. Review the mapping
print("Standardization Mapping:")
print(df_mapping[['Original', 'Standardized']].drop_duplicates().sort_values('Standardized'))

# 7. Apply standardization to all fund DataFrames
updated_funds = 0
for fund in funds_total:
    df = fund.sector_allocation
    if df is not None:
        # Make a copy to avoid SettingWithCopyWarning
        df_copy = df.copy()
        
        # Apply standardization to first column
        original_values = df_copy.iloc[:, 0].copy()
        standardized_values = original_values.map(standardization_dict)
        df_copy.iloc[:, 0] = standardized_values
        df_copy.iloc[:, 0] = (
            df_copy.iloc[:, 0]
            .str.replace(r'\bSector\b', '', regex=True)  # Remove multiple words
            .str.replace(r'\s+', ' ', regex=True)         # Fix extra spaces
            .str.strip()                                 # Remove leading/trailing spaces
        )
        # Update the fund's DataFrame
        fund.sector_allocation = df_copy
        updated_funds += 1

print(f"\n✅ Updated {updated_funds} funds with standardized sector names")

# 8. Verify the results
final_sectors = set()
for fund in funds_total:
    df = fund.sector_allocation
    if df is not None:
        final_sectors.update(df.iloc[:, 0].unique())

print(f"\nBefore standardization: {len(sectors)} unique sectors")
print(f"After standardization: {len(final_sectors)} unique sectors")
print(f"Reduction: {len(sectors) - len(final_sectors)} duplicates removed")

# 9. Show some examples of the changes
print(f"\nExamples of standardization:")
for original, standardized in standardization_dict.items():
    if original != standardized:
        print(f"  '{original}' → '{standardized}'")

Standardization Mapping:
                                Original                         Standardized
153                  Aerospace & Defense                  Aerospace & Defense
57              Aerospace Defense Sector                  Aerospace & Defense
30                                Africa                               Africa
77    Agricultural Farm Machinery Sector   Agricultural Farm Machinery Sector
88          Air Freight Logistics Sector              Air Freight & Logistics
..                                   ...                                  ...
1                              Utilities                            Utilities
110  Vanguard Real Estate II Index Fund1  Vanguard Real Estate II Index Fund1
105                      Water Utilities                      Water Utilities
138               Water Utilities Sector                      Water Utilities
19   Wireless Telecommunication Services  Wireless Telecommunication Services

[161 rows x 2 columns]

✅ Updated 287 

In [45]:
# Get unique values from first column (index 0) of all sector allocation DataFrames
sectors = set()
for fund in funds_total:
    df = fund.sector_allocation
    if df is not None:
        # Get unique values from first column
        unique_values = df.iloc[:, 0].unique()
        sectors.update(unique_values)

print(f"Unique sectors: {len(sectors)}")
print("Sectors:", sorted(sectors))

Unique sectors: 120
Sectors: ['Aerospace & Defense', 'Africa', 'Agricultural Farm Machinery', 'Air Freight & Logistics', 'Asia', 'Automobile Components', 'Automobile Manufacturers', 'Automobiles', 'Banks', 'Basic Materials', 'Beverages', 'Biotechnology', 'Broadline Retail', 'Building Products', 'Capital Markets', 'Chemicals', 'Coal & Consumable Fuels', 'Commercial Services & Supplies', 'Communication Services', 'Communications', 'Communications Equipment', 'Construction & Engineering', 'Construction Machinery Heavy Transportation Equipment', 'Construction Materials', 'Consumer Discretionary', 'Consumer Finance', 'Consumer Staples', 'Consumer Staples Distribution & Retail', 'Containers & Packaging', 'Data Center REITs', 'Distributors', 'Diversified Consumer Services', 'Diversified Telecommunication Services', 'Domestic Equity', 'Domestic Fixed Income', 'Electric Utilities', 'Electrical Components Equipment', 'Electrical Equipment', 'Electronic Components', 'Electronic Equipment, Instrum

### Industry Allocation

In [46]:
from src.simple_rag.utils.chart_utils import validate_and_clean_allocation

for fund in funds_total:
    df = fund.industry_allocation
    if df is not None:
        df = validate_and_clean_allocation(df, 'Industry', sort_by_value=True)
        fund.industry_allocation = df


🔍 Detected header row at index 0
New columns: ['Industry', 'Percent of Total Investments(a)']
VALIDATION REPORT
Category Column: 'Industry' | Value Column: 'Percent of Total Investments(a)'
✅ Row  0: 'Semiconductors & Semiconductor' | '37.9    ' | Numeric: True | Range OK: True
✅ Row  1: 'Machinery                     ' | '18.7    ' | Numeric: True | Range OK: True
✅ Row  2: 'Electrical Equipment          ' | '17.9    ' | Numeric: True | Range OK: True
✅ Row  3: 'Automobiles                   ' | '14.4    ' | Numeric: True | Range OK: True
✅ Row  4: 'Automobile Components         ' | '6.4     ' | Numeric: True | Range OK: True
✅ Row  5: 'Chemicals                     ' | '4.1     ' | Numeric: True | Range OK: True
✅ Row  6: 'Electronic Equipment, Instrume' | '0.6     ' | Numeric: True | Range OK: True

✅ Final: 7 valid rows (sorted by value)
🔍 Detected header row at index 0
New columns: ['Industry', 'Percent of Total  Investments(a)']
VALIDATION REPORT
Category Column: 'Industry' | Val

In [47]:
# Get unique values from first column (index 0) of all sector allocation DataFrames
sectors = set()
for fund in funds_total:
    df = fund.industry_allocation
    if df is not None:
        # Get unique values from first column
        unique_values = df.iloc[:, 0].unique()
        sectors.update(unique_values)

print(f"Unique sectors: {len(sectors)}")
print("Sectors:", sorted(sectors))

Unique sectors: 129
Sectors: ['Aerospace & Defense', 'Alternative Carriers', 'Automobile Components', 'Automobiles', 'Automobiles & Components', 'Banks', 'Biotechnology', 'Brewers', 'Building Products', 'Cable & Satellite', 'Capital Goods', 'Capital Markets', 'Chemicals', 'Coal & Consumable Fuels', 'Commercial & Professional Services', 'Commodity Chemicals', 'Communications Equipment', 'Construction & Engineering', 'Construction Machinery & Heavy Transportation Equipment', 'Construction Materials', 'Consumer Discretionary Distribution & Retail', 'Consumer Durables & Apparel', 'Consumer Finance', 'Consumer Services', 'Consumer Staples Merchandise Retail', 'Containers & Packaging', 'Copper', 'Data Center REITs', 'Distillers & Vintners', 'Diversified Banks', 'Diversified Metals & Mining', 'Diversified REITs', 'Diversified Real Estate Activities', 'Diversified Telecommunication Services', 'Electric Utilities', 'Electrical Components & Equipment', 'Electrical Equipment', 'Electronic Equipme

In [48]:
import pickle
from pathlib import Path

PKL_PATH = Path("./funds_backup_metadata.pkl")
TMP_PATH = PKL_PATH.with_suffix(PKL_PATH.suffix + ".tmp")

with TMP_PATH.open("wb") as f:
    pickle.dump(funds_total, f, protocol=pickle.HIGHEST_PROTOCOL)

TMP_PATH.replace(PKL_PATH)

print(f"Saved {len(funds_total)} funds to pickle file: {PKL_PATH.resolve()}")
print(f"File size: {PKL_PATH.stat().st_size / (1024 * 1024):.2f} MB")

Saved 460 funds to pickle file: /home/luis/Desktop/code/RAG/notebooks/funds_backup_metadata.pkl
File size: 11.19 MB
